# **EE 559 PROJECT**

#### **DATASET** : Student Performance Dataset

#### **TEAM MEMBERS** :

1. Sanjana Vasudeva (USC ID : 7071819723)
2. Vaishnavi Channakeshava (USC ID : 9673718359)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import mutual_info_regression,mutual_info_classif
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

from sklearn.neighbors import NearestCentroid
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold 
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import SGDRegressor
from sklearn import linear_model
from sklearn import  ensemble

In [ ]:
class pre_processing:
  def __init__(self, filename,mission = None):
    self.filename = filename
    self.mission = mission
  
  def preprocess(self, grade):
    data = pd.read_csv(self.filename)
    data_binary = data.copy()
    data_binary.drop(['Mjob', 'Fjob', 'reason', 'guardian'],inplace=True, axis=1)

    data_binary_copy = data_binary.copy()
    
    self.y_data = data_binary_copy[grade]
    data_binary_copy.drop([grade], inplace=True, axis=1)

    if (self.mission == '3'):
      print("mission 3")
      pass
    else:
      data_binary_copy.drop(['G2'], inplace=True, axis=1)
      if grade == 'G1':
          data_binary_copy.drop(['G3'], inplace=True, axis=1)
      else:
          data_binary_copy.drop(['G1'], inplace=True, axis=1)
    
    self.x_data = data_binary_copy

    return self.x_data,self.y_data

  
  def one_hot_encode(self):
    categorical_cols = ['school', 'sex', 'address', 'famsize', 'Pstatus', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic'] 

    data_categorical = self.x_data[categorical_cols]
    le = preprocessing.LabelEncoder()
    data_one_hot = data_categorical.apply(le.fit_transform)
    data_other_cols = self.x_data.drop(columns=categorical_cols)
    self.x_enc = pd.concat([data_one_hot, data_other_cols], axis=1)
    return self.x_enc


  def create_class(self):
    g1_classifier = []
    for i in self.y_data:
      if i<=20 and i>=16:
        g1_classifier.append(1)
      elif i<=15 and i>=14:
        g1_classifier.append(2)
      elif i<=13 and i>=12:
        g1_classifier.append(3)
      elif i<=11 and i>=10:
        g1_classifier.append(4)
      elif i<=9 and i>=0:
        g1_classifier.append(5)

    return g1_classifier

In [ ]:
class feature_selection:
    def __init__(self, xdata, ydata, task = 'regression'):
      self.xdata = xdata
      self.ydata = np.asarray(ydata)
      self.task = task


    def random_forest(self):
      sel = SelectFromModel(RandomForestClassifier(n_estimators = 100))
      sel.fit(self.xdata, self.ydata)
      sel.get_support()
      selected_feat = self.xdata.columns[(sel.get_support())]
      selected_data = self.xdata[selected_feat]
      return selected_feat 

    def mutual_info(self):
      if (self.task == 'regression'):
        selector = SelectKBest(mutual_info_regression, k =10)
        selector.fit(self.xdata, self.ydata)
        selected_feat = self.xdata.columns[selector.get_support()]
        print("mutual",selected_feat)
      else:
        selector = SelectKBest(mutual_info_classif, k =10)
        selector.fit(self.xdata, self.ydata)
        selected_feat = self.xdata.columns[selector.get_support()]
        print("mutual",selected_feat)
      selected_data = self.xdata[selected_feat]
      return selected_feat 

    def chi_squared(self):
      selector = SelectKBest(chi2, k = 16)
      selector.fit(self.xdata, self.ydata)
      selected_feat = self.xdata.columns[selector.get_support()]
      print("chi_squared",selected_feat)
      selected_data = self.xdata[selected_feat]
      return selected_feat 

    def sequential_feature(self):
      sfs_selector = SequentialFeatureSelector(estimator=LinearRegression(), n_features_to_select = 10, cv = 6, direction ='backward')
      sfs_selector.fit(self.xdata,self.ydata)
      col_req_list = self.xdata.columns[sfs_selector.get_support()]
      print("sequential_feature",col_req_list)
      selected_data = self.xdata[col_req_list]
      return col_req_list

    def rfe(self):
      rfe_selector = RFE(estimator=LinearRegression(),n_features_to_select = 15, step = 2)
      rfe_selector.fit(self.xdata,self.ydata)
      col_req_list = self.xdata.columns[rfe_selector.get_support()]
      print("rfe",col_req_list)
      selected_data = self.xdata[col_req_list]
      return col_req_list 

    def pca_features(self):
      pca = PCA()
      X_transformed = pca.fit_transform(self.xdata)
      return pca,X_transformed


    def fisher_features(self):
      lda = LinearDiscriminantAnalysis()
      X_transformed = lda.fit_transform(self.xdata,self.ydata)
      return lda,X_transformed

In [ ]:
class baseline_regression:

    def __init__(self, xtrain,ytrain,xtest,ytest):
      self.xtrain = np.asarray(xtrain)
      self.ytrain = np.asarray(ytrain)
      self.xtest = np.asarray(xtest)
      self.ytest = np.asarray(ytest)


    def trivial_system(self):
      g1_trivial = np.mean(self.ytrain)
      g1_trivial_pred = [g1_trivial]*len(self.ytest)
      mse = mean_squared_error(self.ytest, g1_trivial_pred)
      rmse =  mean_squared_error(self.ytest, g1_trivial_pred, squared=False)
      r2 = r2_score(self.ytest, g1_trivial_pred)    
      return rmse,mse,r2 

    def linear_reg(self):
      reg = LinearRegression().fit(self.xtrain, self.ytrain)
      y_pred_linear_baseline = reg.predict(self.xtest)
      mse = mean_squared_error(self.ytest, y_pred_linear_baseline)
      rmse =  mean_squared_error(self.ytest, y_pred_linear_baseline, squared=False)
      r2 = r2_score(self.ytest, y_pred_linear_baseline)    
      return rmse,mse,r2 


    def one_nn(self):
      nbrs = NearestNeighbors(n_neighbors=1).fit(self.xtrain)
      distances, indices = nbrs.kneighbors(self.xtest)
      y_pred_1nn = self.ytrain[indices]
      mse = mean_squared_error(self.ytest, y_pred_1nn)
      rmse =  mean_squared_error(self.ytest, y_pred_1nn, squared=False)
      r2 = r2_score(self.ytest, y_pred_1nn)    
      return rmse,mse,r2 

In [ ]:
class regression:

  def __init__(self, xtrain,ytrain,xtest,ytest):
    self.xtrain = np.asarray(xtrain)
    self.ytrain = np.asarray(ytrain)
    self.xtest = np.asarray(xtest)
    self.ytest = np.asarray(ytest)

  def svr(self):
    regrassor = SVR(kernel='linear')
    target = self.ytrain.reshape(-1,1)
    regrassor.fit(self.xtrain, target)
    y_pred = regrassor.predict(self.xtest)
    y_pred = y_pred.reshape(-1,1)
    mse = mean_squared_error(self.ytest, y_pred)
    rmse =  mean_squared_error(self.ytest, y_pred, squared=False)
    r2 = r2_score(self.ytest, y_pred)    
    return rmse,mse,r2
  
  def ann(self):
    regr = MLPRegressor(random_state=1, max_iter=500).fit(self.xtrain, self.ytrain)
    y_pred = regr.predict(self.xtest)
    mse = mean_squared_error(self.ytest, y_pred)
    rmse =  mean_squared_error(self.ytest, y_pred, squared=False)
    r2 = r2_score(self.ytest, y_pred)    
    return rmse,mse,r2

  def gradient_boost(self):
    reg = ensemble.GradientBoostingRegressor().fit(self.xtrain, self.ytrain)
    y_pred = reg.predict(self.xtest)
    mse = mean_squared_error(self.ytest, y_pred)
    rmse =  mean_squared_error(self.ytest, y_pred, squared=False)
    r2 = r2_score(self.ytest, y_pred)    
    return rmse,mse,r2

  def knn_regression(self):
    k_list=np.arange(1,80,1)
    knn_dict={}
    knn_list = []
    for i in k_list:
      knn=KNeighborsRegressor(n_neighbors=int(i))
      model_knn=knn.fit(self.xtrain,self.ytrain)
      y_knn_pred=model_knn.predict(self.xtest)
      mse=mean_squared_error(self.ytest,y_knn_pred)
      knn_dict[i]=mse
      knn_list.append(mse)

    min_val = min(knn_list)
    k_size = knn_list.index(min_val)
    knn=KNeighborsRegressor(n_neighbors=k_size)
    model_knn=knn.fit(self.xtrain,self.ytrain)
    y_pred=model_knn.predict(self.xtest)
    mse = mean_squared_error(self.ytest, y_pred)
    rmse =  mean_squared_error(self.ytest, y_pred, squared=False)
    r2 = r2_score(self.ytest, y_pred)    
    return rmse,mse,r2

In [ ]:
class regression_regularization:

  def __init__(self, xtrain,ytrain,xtest,ytest):
    self.xtrain = np.asarray(xtrain)
    self.ytrain = np.asarray(ytrain)
    self.xtest = np.asarray(xtest)
    self.ytest = np.asarray(ytest)

  def svr(self):
    regrassor = SVR(kernel='linear',C=3)
    target = self.ytrain.reshape(-1,1)
    regrassor.fit(self.xtrain, target)
    y_pred = regrassor.predict(self.xtest)
    y_pred = y_pred.reshape(-1,1)
    mse = mean_squared_error(self.ytest, y_pred)
    rmse =  mean_squared_error(self.ytest, y_pred, squared=False)
    r2 = r2_score(self.ytest, y_pred)    
    return rmse,mse,r2
  
  def ann(self):
    regr = MLPRegressor(random_state=1, max_iter=500, alpha = 0.1).fit(self.xtrain, self.ytrain)
    y_pred = regr.predict(self.xtest)
    mse = mean_squared_error(self.ytest, y_pred)
    rmse =  mean_squared_error(self.ytest, y_pred, squared=False)
    r2 = r2_score(self.ytest, y_pred)    
    return rmse,mse,r2

  def ridge_regression(self):
    rr = Ridge(alpha=0.1)
    rr.fit(self.xtrain, self.ytrain) 
    y_pred_lr = rr.predict(self.xtest)
    mse_rr = mean_squared_error(self.ytest, y_pred_lr)
    rmse_rr =  mean_squared_error(self.ytest, y_pred_lr, squared=False)
    r2_rr = r2_score(self.ytest, y_pred_lr) 

    return rmse_rr,mse_rr,r2_rr

  def lasso_regression(self):  
    lr = Lasso(alpha=0.1)
    lr.fit(self.xtrain, self.ytrain) 
    y_pred = lr.predict(self.xtest)
    mse_lr = mean_squared_error(self.ytest, y_pred)
    rmse_lr =  mean_squared_error(self.ytest, y_pred, squared=False)
    r2_lr = r2_score(self.ytest, y_pred) 

    return mse_lr,rmse_lr,r2_lr

In [ ]:
class baseline_classification:

    def __init__(self, xtrain,ytrain,xtest,ytest):
      self.xtrain = np.asarray(xtrain)
      self.ytrain = np.asarray(ytrain)
      self.xtest = np.asarray(xtest)
      self.ytest = np.asarray(ytest)


    def trivial_system(self):
      priors = {}
      priors_1 = 0
      priors_2 = 0
      priors_3 = 0
      priors_4 = 0
      priors_5 = 0

      for i in self.ytrain:
        if i==1:
          priors_1+=1
        elif i==2:
          priors_2+=1
        elif i==3:
          priors_3+=1
        elif i==4:
          priors_4+=1
        else:
          priors_5+=1
  
      priors[1] = priors_1/len(self.ytrain)
      priors[2] = priors_2/len(self.ytrain)
      priors[3] = priors_3/len(self.ytrain)
      priors[4] = priors_4/len(self.ytrain)
      priors[5] = priors_5/len(self.ytrain)

      prior_list = list(priors.values())  
      accuracy = 0
      macro_f1_score = 0

      for k in range(10):
        g1_classifier_trivial = []
        for i in range(len(self.ytest)):
          g1_classifier_trivial.append(np.random.choice(np.arange(1, 6), p=prior_list))

        accuracy += accuracy_score(g1_classifier_trivial, self.ytest)
        macro_f1_score += f1_score(g1_classifier_trivial, self.ytest, average='macro')

      accuracy_avg = accuracy/10
      macro_f1_score_avg = macro_f1_score/10
      cf = confusion_matrix(g1_classifier_trivial, self.ytest)
      return accuracy_avg,macro_f1_score_avg,cf

    

    def nearest_means(self):
      clf = NearestCentroid()
      clf.fit(self.xtrain, self.ytrain)
      baseline_classifier = clf.predict(np.asarray(self.xtest))
      acc = accuracy_score(self.ytest, baseline_classifier)
      macro_f1_score = f1_score(baseline_classifier, self.ytest, average='macro')
      cf = confusion_matrix(baseline_classifier, self.ytest)
      return acc,macro_f1_score,cf

In [ ]:
class classification:

  def __init__(self, xtrain,ytrain,xtest,ytest):
    self.xtrain = np.asarray(xtrain)
    self.ytrain = np.asarray(ytrain)
    self.xtest = np.asarray(xtest)
    self.ytest = np.asarray(ytest)

  def logistic_reg(self):
    clf = LogisticRegression(random_state=0).fit(self.xtrain, self.ytrain)
    pred_class = clf.predict(self.xtest)
    acc = accuracy_score(self.ytest, pred_class)
    macro_f1_score = f1_score(pred_class, self.ytest, average='macro')
    cf = confusion_matrix(pred_class, self.ytest)
    return acc,macro_f1_score,cf

  def svm_classification(self):
    linear = svm.SVC(kernel='linear', C=1, decision_function_shape='ovo').fit(self.xtrain, self.ytrain)
    rbf = svm.SVC(kernel='rbf', gamma=1, C=1, decision_function_shape='ovo').fit(self.xtrain, self.ytrain)
    poly = svm.SVC(kernel='poly', degree=3, C=1, decision_function_shape='ovo').fit(self.xtrain, self.ytrain)
    sig = svm.SVC(kernel='sigmoid', C=1, decision_function_shape='ovo').fit(self.xtrain, self.ytrain)

    linear_pred = linear.predict(self.xtest)
    poly_pred = poly.predict(self.xtest)
    rbf_pred = rbf.predict(self.xtest)
    sig_pred = sig.predict(self.xtest)

    accuracy_list = []
    accuracy_list.append(linear.score(self.xtest, self.ytest))
    accuracy_list.append(poly.score(self.xtest, self.ytest))
    accuracy_list.append(rbf.score(self.xtest, self.ytest))
    accuracy_list.append(sig.score(self.xtest, self.ytest))

    highest_score = max(accuracy_list)
    ind = accuracy_list.index(highest_score)

    if(ind == 0):
      acc = accuracy_score(self.ytest, linear_pred)
      macro_f1_score = f1_score(linear_pred, self.ytest, average='macro')
      cf = confusion_matrix(linear_pred, self.ytest)
      print("Linear Kernel")
      plt.figure(figsize = (7,7))
      sns.heatmap(cf, annot=True, cmap='Blues')
      plt.show()
      return acc,macro_f1_score,cf
    elif(ind==1):
      acc = accuracy_score(self.ytest, poly_pred)
      macro_f1_score = f1_score(poly_pred, self.ytest, average='macro')
      cf = confusion_matrix(poly_pred, self.ytest)
      print("Poly Kernel")
      plt.figure(figsize = (7,7))
      sns.heatmap(cf, annot=True, cmap='Blues')
      plt.show()
      return acc,macro_f1_score,cf
    elif(ind==2):
      acc = accuracy_score(self.ytest, rbf_pred)
      macro_f1_score = f1_score(rbf_pred, self.ytest, average='macro')
      cf = confusion_matrix(rbf_pred, self.ytest)
      print("RBF Kernel")
      plt.figure(figsize = (7,7))
      sns.heatmap(cf, annot=True, cmap='Blues')
      plt.show()
      return acc,macro_f1_score,cf
    else:
      acc = accuracy_score(self.ytest, sig_pred)
      macro_f1_score = f1_score(sig_pred, self.ytest, average='macro')
      cf = confusion_matrix(sig_pred, self.ytest)
      print("Sigmoid Kernel")
      plt.figure(figsize = (7,7))
      sns.heatmap(cf, annot=True, cmap='Blues')
      plt.show()
      return acc,macro_f1_score,cf

  def knn_classification(self):
    # k_list=np.arange(1,80,1)
    k_list =  list(range(1,80))
    knn_dict={} # To store k and mse pairs
    knn_list = []
    for i in k_list:
      knn=KNeighborsClassifier(n_neighbors = int(i))
      model_knn=knn.fit(self.xtrain,self.ytrain)
      y_knn_pred=model_knn.predict(self.xtest)

      mse=mean_squared_error(self.ytest,y_knn_pred)
      knn_dict[i]=mse
      knn_list.append(mse)

    plt.plot(knn_list)
    min_val = min(knn_list)
    neigbor_val = knn_list.index(min_val)
    knn=KNeighborsClassifier(n_neighbors = neigbor_val)
    model_knn=knn.fit(self.xtrain,self.ytrain)
    y_knn_pred=model_knn.predict(self.xtest)
    acc = accuracy_score(self.ytest, y_knn_pred)
    macro_f1_score = f1_score(y_knn_pred, self.ytest, average='macro')
    cf = confusion_matrix(y_knn_pred, self.ytest)
    return acc,macro_f1_score,cf
    

  def gaussian_process_classifier(self):
    kernel = 1.0 * RBF(1.0)
    gpc = GaussianProcessClassifier(kernel=kernel,random_state=0).fit(self.xtrain, self.ytrain)
    x_test_temp = self.xtest.reshape(1,-1)
    y_pred = gpc.predict(self.xtest)
    acc = accuracy_score(self.ytest, y_pred)
    macro_f1_score = f1_score(y_pred, self.ytest, average='macro')
    cf = confusion_matrix(y_pred, self.ytest)
    return acc,macro_f1_score,cf

  def random_forest_classifier(self):
    clf = RandomForestClassifier(max_depth=2, random_state=0)
    clf.fit(self.xtrain, self.ytrain)
    y_pred = clf.predict(self.xtest)
    acc = accuracy_score(self.ytest, y_pred)
    macro_f1_score = f1_score(y_pred, self.ytest, average='macro')
    cf = confusion_matrix(y_pred, self.ytest)
    return acc,macro_f1_score,cf
  
  def mlp_classifier(self):
    clf = MLPClassifier(random_state=1, max_iter=300).fit(self.xtrain, self.ytrain)
    y_pred = clf.predict(self.xtest)
    acc = accuracy_score(self.ytest, y_pred)
    macro_f1_score = f1_score(y_pred, self.ytest, average='macro')
    cf = confusion_matrix(y_pred, self.ytest)
    return acc,macro_f1_score,cf

  def naives_bayesian(self):
    clf = GaussianNB()
    clf.fit(self.xtrain, self.ytrain)
    y_pred = clf.predict(self.xtest)
    acc = accuracy_score(self.ytest, y_pred)
    macro_f1_score = f1_score(y_pred, self.ytest, average='macro')
    cf = confusion_matrix(y_pred, self.ytest)
    return acc,macro_f1_score,cf


In [ ]:
class classification_regularization:

  def __init__(self, xtrain,ytrain,xtest,ytest):
    self.xtrain = np.asarray(xtrain)
    self.ytrain = np.asarray(ytrain)
    self.xtest = np.asarray(xtest)
    self.ytest = np.asarray(ytest)

  def logistic_reg_l1(self):
    clf = LogisticRegression(C=1, penalty='l1', solver='liblinear').fit(self.xtrain, self.ytrain)
    pred_class = clf.predict(self.xtest)
    acc = accuracy_score(self.ytest, pred_class)
    macro_f1_score = f1_score(pred_class, self.ytest, average='macro')
    cf = confusion_matrix(pred_class, self.ytest)
    return acc,macro_f1_score,cf

  
  def logistic_reg_l2(self):
    clf = LogisticRegression(random_state=0,penalty = 'l2').fit(self.xtrain, self.ytrain)
    pred_class = clf.predict(self.xtest)
    acc = accuracy_score(self.ytest, pred_class)
    macro_f1_score = f1_score(pred_class, self.ytest, average='macro')
    cf = confusion_matrix(pred_class, self.ytest)
    return acc,macro_f1_score,cf


  def svm_classification(self):
    linear = svm.SVC(kernel='linear', C=3, decision_function_shape='ovo').fit(self.xtrain, self.ytrain)
    rbf = svm.SVC(kernel='rbf', gamma=1, C=3, decision_function_shape='ovo').fit(self.xtrain, self.ytrain)
    poly = svm.SVC(kernel='poly', degree=3, C=3, decision_function_shape='ovo').fit(self.xtrain, self.ytrain)
    sig = svm.SVC(kernel='sigmoid', C=3, decision_function_shape='ovo').fit(self.xtrain, self.ytrain)

    linear_pred = linear.predict(self.xtest)
    poly_pred = poly.predict(self.xtest)
    rbf_pred = rbf.predict(self.xtest)
    sig_pred = sig.predict(self.xtest)

    accuracy_list = []
    accuracy_list.append(linear.score(self.xtest, self.ytest))
    accuracy_list.append(poly.score(self.xtest, self.ytest))
    accuracy_list.append(rbf.score(self.xtest, self.ytest))
    accuracy_list.append(sig.score(self.xtest, self.ytest))

    highest_score = max(accuracy_list)
    ind = accuracy_list.index(highest_score)

    if(ind == 0):
      acc = accuracy_score(self.ytest, linear_pred)
      macro_f1_score = f1_score(linear_pred, self.ytest, average='macro')
      cf = confusion_matrix(linear_pred, self.ytest)
      print("Linear Kernel")
      plt.figure(figsize = (7,7))
      sns.heatmap(cf, annot=True, cmap='Blues')
      plt.show()
      return acc,macro_f1_score,cf
    elif(ind==1):
      acc = accuracy_score(self.ytest, poly_pred)
      macro_f1_score = f1_score(poly_pred, self.ytest, average='macro')
      cf = confusion_matrix(poly_pred, self.ytest)
      print("Poly Kernel")
      plt.figure(figsize = (7,7))
      sns.heatmap(cf, annot=True, cmap='Blues')
      plt.show()
      return acc,macro_f1_score,cf
    elif(ind==2):
      acc = accuracy_score(self.ytest, rbf_pred)
      macro_f1_score = f1_score(rbf_pred, self.ytest, average='macro')
      cf = confusion_matrix(rbf_pred, self.ytest)
      print("RBF Kernel")
      plt.figure(figsize = (7,7))
      sns.heatmap(cf, annot=True, cmap='Blues')
      plt.show()
      return acc,macro_f1_score,cf
    else:
      acc = accuracy_score(self.ytest, sig_pred)
      macro_f1_score = f1_score(sig_pred, self.ytest, average='macro')
      print("Sigmoid Kernel")
      cf = confusion_matrix(sig_pred, self.ytest)
      plt.figure(figsize = (7,7))
      sns.heatmap(cf, annot=True, cmap='Blues')
      plt.show()
      return acc,macro_f1_score,cf
  
  def mlp_classifier(self):
    clf = MLPClassifier(random_state=1, max_iter=300,alpha = 0.1).fit(self.xtrain, self.ytrain)
    y_pred = clf.predict(self.xtest)
    acc = accuracy_score(self.ytest, y_pred)
    macro_f1_score = f1_score(y_pred, self.ytest, average='macro')
    cf = confusion_matrix(y_pred, self.ytest)
    plt.figure(figsize = (7,7))
    sns.heatmap(cf, annot=True, cmap='Blues')
    plt.show()
    return acc,macro_f1_score,cf

In [ ]:
class regression_analysis:

  def __init__(self,x_train,y_train,x_test,y_test):
    self.x_train = x_train
    self.y_train = y_train
    self.x_test = x_test
    self.y_test = y_test

  
  def perform_regression(self):

    reg_model = regression(self.x_train,self.y_train,self.x_test,self.y_test)

    rmse_svr,mse_svr,r2_svr = reg_model.svr()
    print("Performance of SVR")
    print("rmse of SVR ", rmse_svr)
    print("mse of SVR ", mse_svr)
    print("r2 score of SVR ", r2_svr)
    print("\n")

    #ANN

    rmse_ann,mse_ann,r2_ann = reg_model.ann()
    print("Performance of ANN")
    print("rmse of ANN ", rmse_ann,)
    print("mse of ANN ", mse_ann)
    print("r2 score of ANN ", r2_ann)
    print("\n")

    #Gradient boost regressor

    rmse_lr,mse_lr,r2_lr = reg_model.gradient_boost()
    print("Performance of Gradient boost regressor")
    print("rmse of Gradient boost regressorn ", rmse_lr,)
    print("mse of Gradient boost regressor ", mse_lr)
    print("r2 score of Gradient boost regressor ", r2_lr)
    print("\n")


    #knn regression
    rmse_knn,mse_knn,r2_knn = reg_model.knn_regression()
    print("Performance of KNN")
    print("rmse of KNN ", rmse_knn,)
    print("mse of KNN ", mse_knn)
    print("r2 score of KNN", r2_knn)
    print("\n")

  def perform_regression_regularization(self):

    reg_model = regression_regularization(self.x_train,self.y_train,self.x_test,self.y_test)

    rmse_svr,mse_svr,r2_svr = reg_model.svr()
    print("Performance of SVR")
    print("rmse of SVR ", rmse_svr)
    print("mse of SVR ", mse_svr)
    print("r2 score of SVR ", r2_svr)
    print("\n")

    #ANN
    print("Performance of ANN")
    rmse_ann,mse_ann,r2_ann = reg_model.ann()
    print("rmse of ANN ", rmse_ann,)
    print("mse of ANN ", mse_ann)
    print("r2 score of ANN ", r2_ann)
    print("\n")

    #Ridge regression
    print("Performance of Ridge Regression")
    rmse_lr,mse_lr,r2_lr = reg_model.ridge_regression()
    print("rmse of ridge regression ", rmse_lr,)
    print("mse of ridge regression ", mse_lr)
    print("r2 score of ridge regression ", r2_lr)
    print("\n")


    #Lasso regression
    print("Performance of Lasso Regression")
    rmse_knn,mse_knn,r2_knn = reg_model.lasso_regression()
    print("rmse of lasso regression", rmse_knn,)
    print("mse of lasso regression ", mse_knn)
    print("r2 score of lasso regression", r2_knn)
    print("\n")

In [ ]:
def regresion_feature_selection(x_train,y_train,x_test,y_test,flag = 'regularize',mission = None):

    feature_selected = feature_selection(x_train,y_train)

    fs_random = feature_selected.random_forest()
    fs_mututal = feature_selected.mutual_info()
    fs_chi = feature_selected.chi_squared()
    fs_seq = feature_selected.sequential_feature()
    fs_rfe = feature_selected.rfe()
    pca_g1, x_train_tranformed_pca = feature_selected.pca_features()
    x_test_tranformed = pca_g1.transform(x_test)


    print("The performance of different regression models on features selected using random forest")
    reg_analysis = regression_analysis(x_train[fs_random],y_train,x_test[fs_random],y_test)
    if (flag=='regularize'):
      reg_analysis.perform_regression_regularization()
    else:
      reg_analysis.perform_regression()
    print("\n")

  
    print("The performance of different regression models on features selected using Mutual Information")
    reg_analysis = regression_analysis(x_train[fs_mututal],y_train,x_test[fs_mututal],y_test)
    if (flag=='regularize'):
      reg_analysis.perform_regression_regularization()
    else:
      reg_analysis.perform_regression()
    print("\n")
    
    print("The performance of different regression models on features selected using Chi squared")
    reg_analysis = regression_analysis(x_train[fs_chi],y_train,x_test[fs_chi],y_test)
    if (flag=='regularize'):
      reg_analysis.perform_regression_regularization()
      
    else:
      reg_analysis.perform_regression()
    print("\n")
  

    print("The performance of different regression models on features selected using Sequential Features")
    reg_analysis = regression_analysis(x_train[fs_seq],y_train,x_test[fs_seq],y_test)
    if (flag=='regularize'):
       reg_analysis.perform_regression_regularization()
    else:
      reg_analysis.perform_regression()
    print("\n")
  
    print("The performance of different regression models on features selected using RFE")
    reg_analysis = regression_analysis(x_train[fs_rfe],y_train,x_test[fs_rfe],y_test)
    if (flag=='regularize'):
      reg_analysis.perform_regression_regularization()
    else:
      reg_analysis.perform_regression()
    print("\n")


    if(mission !=3):
      print("The performance of different regression models on features selected using PCA")
      reg_analysis = regression_analysis(x_train_tranformed_pca,y_train,x_test_tranformed,y_test)
      if (flag=='regularize'):
        reg_analysis.perform_regression_regularization()
      else:
        reg_analysis.perform_regression()
    print("\n")

In [ ]:
class classification_analysis:

  def __init__(self,x_train,y_train,x_test,y_test):
    self.x_train = x_train
    self.y_train = y_train
    self.x_test = x_test
    self.y_test = y_test

  def perform_classification(self):
    class_model = classification(self.x_train,self.y_train,self.x_test,self.y_test)

    acc,macro_f1_score,cf = class_model.logistic_reg()
    print("Accuracy of logistic regression system ", acc)
    print("Macro f1 score of logistic regression system ", macro_f1_score)
    print("Confusion matrix of logistic regression system : \n", cf)
    print("\n")

    acc,macro_f1_score,cf = class_model.svm_classification()
    print("Accuracy of SVM system ", acc)
    print("Macro f1 score of SVM system ", macro_f1_score)
    print("Confusion matrix of SVM system : \n", cf)
    print("\n")

    acc,macro_f1_score,cf = class_model.gaussian_process_classifier()
    print("Accuracy of Gaussian process classifier ", acc)
    print("Macro f1 score of Gaussian process classifier", macro_f1_score)
    print("Confusion matrix of Gaussian process classifier : \n", cf)  
    print("\n")

    acc,macro_f1_score,cf =  class_model.random_forest_classifier()
    print("Accuracy of random forest classifier ", acc)
    print("Macro f1 score of random forest classifier", macro_f1_score)
    print("Confusion matrix of random forest classifier : \n", cf)  
    print("\n")

    acc,macro_f1_score,cf =  class_model.mlp_classifier()
    print("Accuracy of MLP classifier ", acc)
    print("Macro f1 score of MLP classifier", macro_f1_score)
    print("Confusion matrix of MLP classifier : \n", cf)  
    print("\n")

    acc,macro_f1_score,cf =  class_model.naives_bayesian()
    print("Accuracy of naives bayesian classifier ", acc)
    print("Macro f1 score of naives bayesian classifier", macro_f1_score)
    print("Confusion matrix of naives bayesian classifier : \n", cf) 
    print("\n")

    acc,macro_f1_score,cf = class_model.knn_classification()
    print("Accuracy of KNN system ", acc)
    print("Macro f1 score of KNN system ", macro_f1_score)
    print("Confusion matrix of KNN system : \n", cf)   
    print("\n")


  def perform_classification_regularisation(self):
    class_model = classification_regularization(self.x_train,self.y_train,self.x_test,self.y_test)

    acc,macro_f1_score,cf = class_model.logistic_reg_l1()
    print("Accuracy of logistic regression system ", acc)
    print("Macro f1 score of logistic regression system ", macro_f1_score)
    print("Confusion matrix of logistic regression system : \n", cf)
    print("\n")

    acc,macro_f1_score,cf = class_model.logistic_reg_l2()
    print("Accuracy of logistic regression system ", acc)
    print("Macro f1 score of logistic regression system ", macro_f1_score)
    print("Confusion matrix of logistic regression system : \n", cf)
    print("\n")

    acc,macro_f1_score,cf = class_model.svm_classification()
    print("Accuracy of SVM system ", acc)
    print("Macro f1 score of SVM system ", macro_f1_score)
    print("Confusion matrix of SVM system : \n", cf)  
    print("\n")

    acc,macro_f1_score,cf =  class_model.mlp_classifier()
    print("Accuracy of MLP classifier ", acc)
    print("Macro f1 score of MLP classifier", macro_f1_score)
    print("Confusion matrix of MLP classifier : \n", cf)   
    print("\n")

In [ ]:
def classification_feature_selection(x_train,y_train,x_test,y_test,flag = 'regularize',mission = None):

    feature_selected = feature_selection(x_train,y_train,task='classification')

    fs_random = feature_selected.random_forest()
    fs_mututal = feature_selected.mutual_info()
    fs_chi = feature_selected.chi_squared()
    fs_seq = feature_selected.sequential_feature()
    fs_rfe = feature_selected.rfe()
    pca_g1, x_train_tranformed_pca = feature_selected.pca_features()
    x_test_tranformed = pca_g1.transform(x_test)
    lda_g1, x_train_tranformed_lda = feature_selected.fisher_features()
    x_test_tranformed_lda = lda_g1.transform(x_test)


    print("The performance of different classification models on features selected using random forest")
    class_analysis = classification_analysis(x_train[fs_random],y_train,x_test[fs_random],y_test)
    if (flag=='regularize'):
      class_analysis.perform_classification_regularisation()
    else:
      class_analysis.perform_classification()
    print("\n")
  
    print("The performance of different classification models on features selected using Mutual Information")
    class_analysis = classification_analysis(x_train[fs_mututal],y_train,x_test[fs_mututal],y_test)
    if (flag=='regularize'):
      class_analysis.perform_classification_regularisation()
    else:
      class_analysis.perform_classification()
    print("\n")
    
    print("The performance of different classification models on features selected using Chi squared")
    class_analysis = classification_analysis(x_train[fs_chi],y_train,x_test[fs_chi],y_test)
    if (flag=='regularize'):
      class_analysis.perform_classification_regularisation()
    else:
      class_analysis.perform_classification()
    print("\n")

    print("The performance of different classification models on features selected using Sequential Features")
    class_analysis = classification_analysis(x_train[fs_seq],y_train,x_test[fs_seq],y_test)
    if (flag=='regularize'):
       class_analysis.perform_classification_regularisation()
    else:
      class_analysis.perform_classification()
    print("\n")
  
    print("The performance of different classification models on features selected using RFE")
    class_analysis = classification_analysis(x_train[fs_rfe],y_train,x_test[fs_rfe],y_test)
    if (flag=='regularize'):
      class_analysis.perform_classification_regularisation()
    else:
      class_analysis.perform_classification()
    print("\n")

    if(mission!=3):
      print("The performance of different classification models on features selected using PCA")
      class_analysis = classification_analysis(x_train_tranformed_pca,y_train,x_test_tranformed,y_test)
      if (flag=='regularize'):
        class_analysis.perform_classification_regularisation()
      else:
        class_analysis.perform_classification()
    print("\n")

    if(mission!=3):
      print("The performance of different classification models on features selected using FLD")
      class_analysis = classification_analysis(x_train_tranformed_lda,y_train,x_test_tranformed_lda,y_test)
      if (flag=='regularize'):
        class_analysis.perform_classification_regularisation()
      else:
        class_analysis.perform_classification()
    print("\n")

In [ ]:
#k fold

class k_fold_validation:
    def __init__(self, xtrain,ytrain,xtest,ytest):
      self.xtrain = xtrain
      self.ytrain = ytrain
      self.xtest = xtest
      self.ytest = ytest

    def k_fold_regression(self):

      k = 5
      kf = KFold(n_splits=k, random_state=None)

      rmse_svr_list = []
      rmse_ann_list = []
      rmse_lr_list = []
      rmse_knn_list = []

      mse_svr_list = []
      mse_ann_list = []
      mse_lr_list = []
      mse_knn_list = []

      r2_svr_list = []
      r2_ann_list = []
      r2_lr_list = []
      r2_knn_list = []

      
      for train_index , test_index in kf.split(self.xtrain):
          X_train , X_test = self.xtrain.iloc[train_index,:],self.xtrain.iloc[test_index,:]
          y_train , y_test = self.ytrain[train_index],self.ytrain[test_index]
          regression_model = regression(X_train,y_train,X_test,y_test)
          rmse_svr,mse_svr,r2_svr = regression_model.svr()
          rmse_ann,mse_ann,r2_ann = regression_model.ann()
          rmse_lr,mse_lr,r2_lr = regression_model.gradient_boost()
          rmse_knn,mse_knn,r2_knn = regression_model.knn_regression()

          mse_svr_list.append(mse_svr)
          mse_ann_list.append(mse_ann)
          mse_lr_list.append(mse_lr)
          mse_knn_list.append(mse_knn)

          rmse_svr_list.append(rmse_svr)
          rmse_ann_list.append(rmse_ann)
          rmse_lr_list.append(rmse_lr)
          rmse_knn_list.append(rmse_knn)

          r2_svr_list.append(r2_svr)
          r2_ann_list.append(r2_ann)
          r2_lr_list.append(r2_lr)
          r2_knn_list.append(r2_knn)

      
      avg_mse_svr = sum(mse_svr_list)/k
      avg_mse_ann = sum(mse_ann_list)/k
      avg_mse_lr = sum(mse_lr_list)/k
      avg_mse_knn = sum(mse_knn_list)/k

      avg_rmse_svr = sum(rmse_svr_list)/k
      avg_rmse_ann = sum(rmse_ann_list)/k
      avg_rmse_lr = sum(rmse_lr_list)/k
      avg_rmse_knn = sum(rmse_knn_list)/k

      avg_r2_svr = sum(r2_svr_list)/k
      avg_r2_ann = sum(r2_ann_list)/k
      avg_r2_lr = sum(r2_lr_list)/k
      avg_r2_knn = sum(r2_knn_list)/k


      print('Avg MSE, RMSE and r2 score of SVR',avg_mse_svr,avg_rmse_svr,avg_r2_svr)
      print('Avg MSE, RMSE and r2 score of ANN',avg_mse_ann,avg_rmse_ann,avg_r2_ann)
      print('Avg MSE, RMSE and r2 score of Gradient boost regressor',avg_mse_lr,avg_rmse_lr,avg_r2_lr)
      print('Avg MSE, RMSE and r2 score of KNN',avg_mse_knn,avg_rmse_knn,avg_r2_knn)
      print("\n")


    def k_fold_regression_regularize(self):

      k = 5
      kf = KFold(n_splits=k, random_state=None)

      rmse_svr_list = []
      rmse_ann_list = []
      rmse_ridge_list = []
      rmse_lasso_list = []

      mse_svr_list = []
      mse_ann_list = []
      mse_ridge_list = []
      mse_lasso_list = []

      r2_svr_list = []
      r2_ann_list = []
      r2_ridge_list = []
      r2_lasso_list = []

      
      for train_index , test_index in kf.split(self.xtrain):
          X_train , X_test = self.xtrain.iloc[train_index,:],self.xtrain.iloc[test_index,:]
          y_train , y_test = self.ytrain[train_index],self.ytrain[test_index]
      
          regression_model = regression_regularization(X_train,y_train,X_test,y_test)
          rmse_svr,mse_svr,r2_svr = regression_model.svr()
          rmse_ann,mse_ann,r2_ann = regression_model.ann()
          rmse_ridge,mse_ridge,r2_ridge = regression_model.ridge_regression()
          rmse_lasso,mse_lasso,r2_lasso = regression_model.lasso_regression()

          mse_svr_list.append(mse_svr)
          mse_ann_list.append(mse_ann)
          mse_ridge_list.append(mse_ridge)
          mse_lasso_list.append(mse_lasso)

          rmse_svr_list.append(rmse_svr)
          rmse_ann_list.append(rmse_ann)
          rmse_ridge_list.append(rmse_ridge)
          rmse_lasso_list.append(rmse_lasso)


          r2_svr_list.append(r2_svr)
          r2_ann_list.append(r2_ann)
          r2_ridge_list.append(r2_ridge)
          r2_lasso_list.append(r2_lasso)

      
      avg_mse_svr = sum(mse_svr_list)/k
      avg_mse_ann = sum(mse_ann_list)/k
      avg_mse_ridge = sum(mse_ridge_list)/k
      avg_mse_lasso = sum(mse_lasso_list)/k

      avg_rmse_svr = sum(rmse_svr_list)/k
      avg_rmse_ann = sum(rmse_ann_list)/k
      avg_rmse_ridge = sum(rmse_ridge_list)/k
      avg_rmse_lasso = sum(rmse_lasso_list)/k

      avg_r2_svr = sum(r2_svr_list)/k
      avg_r2_ann = sum(r2_ann_list)/k
      avg_r2_ridge = sum(r2_ridge_list)/k
      avg_r2_lasso = sum(r2_lasso_list)/k


      print('Avg MSE, RMSE and r2 score of SVR',avg_mse_svr,avg_rmse_svr,avg_r2_svr)
      print('Avg MSE, RMSE and r2 score of ANN',avg_mse_ann,avg_rmse_ann,avg_r2_ann)
      print('Avg MSE, RMSE and r2 score of Ridge regression',avg_mse_ridge,avg_rmse_ridge,avg_r2_ridge)
      print('Avg MSE, RMSE and r2 score of Lasso regression',avg_mse_lasso,avg_rmse_lasso,avg_r2_lasso)
      print("\n")

    def k_fold_classification(self):

      k = 5
      kf = KFold(n_splits=k, random_state=None)

      acc_lr_list = []
      acc_svm_list = []
      acc_knn_list = []
      acc_gau_list = []
      acc_rf_list = []
      acc_mlp_list = []
      acc_nb_list = []

      macro_f1_lr_list = []
      macro_f1_svm_list = []
      macro_f1_knn_list = []
      macro_f1_gau_list = []
      macro_f1_rf_list = []
      macro_f1_mlp_list = []
      macro_f1_nb_list = []

      
      for train_index , test_index in kf.split(self.xtrain):
          X_train , X_test = self.xtrain.iloc[train_index,:],self.xtrain.iloc[test_index,:]
          y_train , y_test = self.ytrain[train_index],self.ytrain[test_index]
          classification_model = classification(X_train,y_train,X_test,y_test)

          acc_lr,macro_f1_lr,_ = classification_model.logistic_reg()
          acc_svm,macro_f1_svm,_ = classification_model.svm_classification()
          acc_knn,macro_f1_knn,_ = classification_model.knn_classification()
          acc_gau,macro_f1_gau,_ = classification_model.gaussian_process_classifier()
          acc_rf,macro_f1_rf,_ = classification_model.random_forest_classifier()
          acc_mlp,macro_f1_mlp,_ = classification_model.mlp_classifier()
          acc_nb,macro_f1_nb,_ = classification_model.naives_bayesian()

          acc_lr_list.append(acc_lr)
          acc_svm_list.append(acc_svm)
          acc_knn_list.append(acc_knn)
          acc_gau_list.append(acc_gau)
          acc_rf_list.append(acc_rf)
          acc_mlp_list.append(acc_mlp)
          acc_nb_list.append(acc_nb)

          macro_f1_lr_list.append(macro_f1_lr)
          macro_f1_svm_list.append(macro_f1_svm)
          macro_f1_knn_list.append(macro_f1_knn)
          macro_f1_gau_list.append(macro_f1_gau)
          macro_f1_rf_list.append(macro_f1_rf)
          macro_f1_mlp_list.append(macro_f1_mlp)
          macro_f1_nb_list.append(macro_f1_nb)

      
      avg_acc_lr = sum(acc_lr_list)/k
      avg_acc_svm = sum(acc_svm_list)/k
      avg_acc_knn = sum(acc_knn_list)/k
      avg_acc_gau = sum(acc_gau_list)/k
      avg_acc_rf = sum(acc_rf_list)/k
      avg_acc_mlp = sum(acc_mlp_list)/k
      avg_acc_nb = sum(acc_nb_list)/k


      avg_macro_f1_lr = sum(macro_f1_lr_list)/k
      avg_macro_f1_svm = sum(macro_f1_svm_list)/k
      avg_macro_f1_knn = sum(macro_f1_knn_list)/k
      avg_macro_f1_gau = sum(macro_f1_gau_list)/k
      avg_amacro_f1_rf = sum(macro_f1_rf_list)/k
      avg_macro_f1_mlp = sum(macro_f1_mlp_list)/k
      avg_macro_f1_nb = sum(macro_f1_nb_list)/k


      print('Avg accuracy and macro f1 score of Logistic regression',avg_acc_lr,avg_macro_f1_lr)
      print('Avg accuracy and macro f1 score of SVM',avg_acc_svm,avg_macro_f1_svm)
      print('Avg accuracy and macro f1 score of KNN',avg_acc_knn,avg_macro_f1_knn)
      print('Avg accuracy and macro f1 score of Gaussian Process classifier',avg_acc_gau,avg_macro_f1_gau)
      print('Avg accuracy and macro f1 score of Random Forest',avg_acc_rf,avg_amacro_f1_rf)
      print('Avg accuracy and macro f1 score of MLP',avg_acc_mlp,avg_macro_f1_mlp)
      print('Avg accuracy and macro f1 score of Naives Bayesian',avg_acc_nb,avg_macro_f1_nb)
      print("\n")

    def k_fold_classification_regularize(self):

      k = 5
      kf = KFold(n_splits=k, random_state=None)

      acc_l1_list = []
      acc_l2_list = []
      acc_svm_list = []
      acc_mlp_list = []

      macro_f1_l1_list = []
      macro_f1_l2_list = []
      macro_f1_svm_list = []
      macro_f1_mlp_list = []

      
      for train_index , test_index in kf.split(self.xtrain):
          X_train , X_test = self.xtrain.iloc[train_index,:],self.xtrain.iloc[test_index,:]
          y_train , y_test = self.ytrain[train_index],self.ytrain[test_index]
      
          classification_model = classification_regularization(X_train,y_train,X_test,y_test)

          acc_l1,macro_f1_l1,_ = classification_model.logistic_reg_l1()
          acc_l2,macro_f1_l2,_ = classification_model.logistic_reg_l2()
          acc_svm,macro_f1_svm,_ = classification_model.svm_classification()
          acc_mlp,macro_f1_mlp,_ = classification_model.mlp_classifier()


          acc_l1_list.append(acc_l1)
          acc_l2_list.append(acc_l2)
          acc_svm_list.append(acc_svm)
          acc_mlp_list.append(acc_mlp)

          macro_f1_l1_list.append(macro_f1_l1)
          macro_f1_l2_list.append(macro_f1_l2)
          macro_f1_svm_list.append(macro_f1_svm)
          macro_f1_mlp_list.append(macro_f1_mlp)

      
      avg_acc_l1 = sum(acc_l1_list)/k
      avg_acc_l2 = sum(acc_l2_list)/k
      avg_acc_svm = sum(acc_svm_list)/k
      avg_acc_mlp = sum(acc_mlp_list)/k

      avg_macro_f1_l1 = sum(macro_f1_l1_list)/k
      avg_macro_f1_l2 = sum(macro_f1_l2_list)/k
      avg_macro_f1_svm = sum(macro_f1_svm_list)/k
      avg_macro_f1_mlp = sum(macro_f1_mlp_list)/k


      print('Avg accuracy and macro f1 score of Logistic regression L1',avg_acc_l1,avg_macro_f1_l1)
      print('Avg accuracy and macro f1 score of Logistic regression L2',avg_acc_l2,avg_macro_f1_l2)
      print('Avg accuracy and macro f1 score of SVM',avg_acc_svm,avg_macro_f1_svm)
      print('Avg accuracy and macro f1 score of MLP',avg_acc_mlp,avg_macro_f1_mlp)
      print("\n")

In [ ]:
def regression_kfold(x_train,y_train,x_test,y_test,flag='regularize'):

  feature_selected = feature_selection(x_train,y_train)

  fs_random = feature_selected.random_forest()
  print("The performance of different regression models on features selected using random forest")
  val = k_fold_validation(x_train[fs_random],y_train,x_test[fs_random],y_test)
  if flag == 'regularize':
    val.k_fold_regression_regularize()
  else:
    val.k_fold_regression()
  print("\n")

  fs_mututal = feature_selected.mutual_info()
  print("The performance of different regression models on features selected using Mutual Information")
  val = k_fold_validation(x_train[fs_mututal],y_train,x_test[fs_mututal],y_test)
  if flag == 'regularize':
    val.k_fold_regression_regularize()
  else:
     val.k_fold_regression()
  print("\n")

  fs_chi = feature_selected.chi_squared()
  print("The performance of different regression models on features selected using Chi squared")
  val = k_fold_validation(x_train[fs_chi],y_train,x_test[fs_chi],y_test)
  if flag == 'regularize':
    val.k_fold_regression_regularize()
  else:
    val.k_fold_regression()
  print("\n")

  fs_seq = feature_selected.sequential_feature()
  print("The performance of different regression models on features selected using Sequential Features")
  val = k_fold_validation(x_train[fs_seq],y_train,x_test[fs_seq],y_test)
  if flag == 'regularize':
    val.k_fold_regression_regularize()
  else:
    val.k_fold_regression()
  print("\n")


  fs_rfe = feature_selected.rfe()
  print("The performance of different regression models on features selected using RFE")
  val = k_fold_validation(x_train[fs_rfe],y_train,x_test[fs_rfe],y_test)
  if flag == 'regularize':
    val.k_fold_regression_regularize()
  else:
    val.k_fold_regression()
  print("\n")

In [ ]:
def classification_kfold(x_train,y_train,x_test,y_test,flag='regularize'):

  feature_selected = feature_selection(x_train,y_train,task='classification')

  fs_random = feature_selected.random_forest()
  print("The performance of different regression models on features selected using random forest")
  val = k_fold_validation(x_train[fs_random],y_train,x_test[fs_random],y_test)
  if flag == 'regularize':
    val.k_fold_classification_regularize()
  else:
     val.k_fold_classification()
  print("\n")

  fs_mututal = feature_selected.mutual_info()
  print("The performance of different regression models on features selected using Mutual Information")
  val = k_fold_validation(x_train[fs_mututal],y_train,x_test[fs_mututal],y_test)
  if flag == 'regularize':
    val.k_fold_classification_regularize()
  else:
     val.k_fold_classification()
  print("\n")

  fs_chi = feature_selected.chi_squared()
  print("The performance of different regression models on features selected using Chi squared")
  val = k_fold_validation(x_train[fs_chi],y_train,x_test[fs_chi],y_test)
  if flag == 'regularize':
    val.k_fold_classification_regularize()
  else:
     val.k_fold_classification()
  print("\n")

  fs_seq = feature_selected.sequential_feature()
  print("The performance of different regression models on features selected using Sequential Features")
  val = k_fold_validation(x_train[fs_seq],y_train,x_test[fs_seq],y_test)
  if flag == 'regularize':
    val.k_fold_classification_regularize()
  else:
     val.k_fold_classification()
  print("\n")

  fs_rfe = feature_selected.rfe()
  print("The performance of different regression models on features selected using RFE")
  val = k_fold_validation(x_train[fs_rfe],y_train,x_test[fs_rfe],y_test)
  if flag == 'regularize':
    val.k_fold_classification_regularize()
  else:
     val.k_fold_classification()
  print("\n")
  

In [ ]:
class performance_regression:
  def __init__(self,train_data_path,test_data_path,grade,mission=None):
    self.train_data_path = train_data_path
    self.test_data_path = test_data_path
    self.grade = grade
    self.mission = mission

  def data_preprocess(self):
    pre_proessing_train = pre_processing(self.train_data_path,mission=self.mission)
    pre_proessing_test = pre_processing(self.test_data_path,mission=self.mission)

    _,self.y_train = pre_proessing_train.preprocess(self.grade)
    self.x_train = pre_proessing_train.one_hot_encode()
    self.y_train_class = pre_proessing_train.create_class() 
    self.y_train_classifier = self.y_train_class
    _,self.y_test = pre_proessing_test.preprocess(self.grade)
    self.x_test = pre_proessing_test.one_hot_encode()
    self.y_test_class = pre_proessing_test.create_class() 
    self.y_test_classifier = self.y_test_class

  def reference_systems(self):
    #baseline
    print("PERFORMANCE OF REFERENCE SYSTEMS")

    baseline_reg = baseline_regression(self.x_train,self.y_train,self.x_test,self.y_test)

    #trivial system
    rmse_ts,mse_ts,r2_ts = baseline_reg.trivial_system()
    print("Performance of Trivial Systems")
    print("rmse of trivial system ", rmse_ts)
    print("mse of trivial system ", mse_ts)
    print("r2 score of trivial system ", r2_ts)
    print("\n")

    #linear regressor
    rmse_lr,mse_lr,r2_lr = baseline_reg.linear_reg()
    print("Performance of Linear Regressor")
    print("rmse of linear regressor ", rmse_lr)
    print("mse of linear regressor ", mse_lr)
    print("r2 score of linear regressor ", r2_lr)
    print("\n")

    #one_nn
    rmse_1nn,mse_1nn,r2_1nn = baseline_reg.one_nn()
    print("Performance of 1NN")
    print("rmse of 1nn ", rmse_1nn)
    print("mse of 1nn ", mse_1nn)
    print("r2 score of 1nn ", r2_1nn)
    print("\n")


  def regression_systems(self,performance = None):
  
    #PERFORMANCE OF THE REGRESSION SYSTEMS WITHOUT FEATURE SELECTION
    print("PERFORMANCE OF THE REGRESSION SYSTEMS WITHOUT FEATURE SELECTION")
    reg_model = regression_analysis(self.x_train,self.y_train,self.x_test,self.y_test)
    reg_model.perform_regression()

    #PERFORMANCE OF THE REGRESSION SYSTEMS WITH FEATURE SELECTION
    print("PERFORMANCE OF THE REGRESSION SYSTEMS WITH FEATURE SELECTION")
    regresion_feature_selection(self.x_train,self.y_train,self.x_test,self.y_test,flag = None)

    #PERFORMANCE OF THE REGRESSION SYSTEMS WITHOUT FEATURE SELECTION WITH NORMALISATION
    print("PERFORMANCE OF THE REGRESSION SYSTEMS WITHOUT FEATURE SELECTION WITH NORMALISATION")
    scaler = preprocessing.MinMaxScaler()
    self.x_train_normalised = self.x_train.copy()
    self.x_test_normalised  = self.x_test.copy()
    self.x_train_normalised.iloc[:]= scaler.fit_transform(self.x_train.to_numpy())
    self.x_test_normalised.iloc[:]  = scaler.transform(self.x_test.to_numpy())

    reg_model = regression_analysis(self.x_train_normalised,self.y_train,self.x_test_normalised,self.y_test)
    reg_model.perform_regression()

    #PERFORMANCE OF THE REGRESSION SYSTEMS WITH FEATURE SELECTION WITH NORMALISATION
    print("PERFORMANCE OF THE REGRESSION SYSTEMS WITH FEATURE SELECTION WITH NORMALISATION")
    regresion_feature_selection(self.x_train_normalised,self.y_train,self.x_test_normalised,self.y_test,flag = None,mission = self.mission)


    if(performance == 'Normalisation'):
      print("PERFORMANCE OF THE REGRESSION SYSTEMS WITH AND WITHOUT FEATURE SELECTION WITH NORMALISATION WITH REGULARIZATION")
      #PERFORMANCE OF THE REGRESSION SYSTEMS WITHOUT FEATURE SELECTION WITH NORMALISATION WITH REGULARIZATION
      print("------------------")
      reg_model = regression_analysis(self.x_train_normalised,self.y_train,self.x_test_normalised,self.y_test)
      reg_model.perform_regression_regularization()

      #PERFORMANCE OF THE REGRESSION SYSTEMS WITH FEATURE SELECTION WITH NORMALISATION WITH REGULARIZATION
      regresion_feature_selection(self.x_train_normalised,self.y_train,self.x_test_normalised,self.y_test,mission = self.mission)

      print("----------------")
      print("PERFORMANCE OF THE REGRESSION SYSTEMS WITH AND WITHOUT FEATURE SELECTION WITH NORMALISATION WITH VALIDATION")
      #PERFORMANCE OF THE REGRESSION SYSTEMS WITHOUT FEATURE SELECTION WITH NORMALISATION WITH VALIDATION
      val = k_fold_validation(self.x_train_normalised,self.y_train,self.x_test_normalised,self.y_test)
      val.k_fold_regression()

      #PERFORMANCE OF THE REGRESSION SYSTEMS WITH FEATURE SELECTION WITH NORMALISATION WITH VALIDATION
      regression_kfold(self.x_train_normalised,self.y_train,self.x_test_normalised,self.y_test,flag=None)

      print("----------------")
      print("PERFORMANCE OF THE REGRESSION SYSTEMS WITH AND WITHOUT FEATURE SELECTION WITH NORMALISATION WITH VALIDATION AND REGULARIZATION")

      #PERFORMANCE OF THE REGRESSION SYSTEMS WITHOUT FEATURE SELECTION WITH NORMALISATION WITH VALIDATION WITH REGULARISATION
      val = k_fold_validation(self.x_train_normalised,self.y_train,self.x_test_normalised,self.y_test)
      val.k_fold_regression_regularize()

      #PERFORMANCE OF THE REGRESSION SYSTEMS WITH FEATURE SELECTION WITHOUT NORMALISATION WITH VALIDATION WITH REGULARISATION
      regression_kfold(self.x_train_normalised,self.y_train,self.x_test_normalised,self.y_test)

    else:
      #PERFORMANCE OF THE REGRESSION SYSTEMS WITHOUT FEATURE SELECTION WITH REGULARIZATION
      print("PERFORMANCE OF THE REGRESSION SYSTEMS WITH AND WITHOUT FEATURE SELECTION WITH REGULARIZATION")
      reg_model = regression_analysis(self.x_train,self.y_train,self.x_test,self.y_test)
      reg_model.perform_regression_regularization()

      #PERFORMANCE OF THE REGRESSION SYSTEMS WITH FEATURE SELECTION WITH NORMALISATION WITH REGULARIZATION
      regresion_feature_selection(self.x_train,self.y_train,self.x_test,self.y_test,mission = self.mission)

      print("----------------")
      print("PERFORMANCE OF THE REGRESSION SYSTEMS WITH AND WITHOUT FEATURE SELECTION WITH VALIDATION")
      #PERFORMANCE OF THE REGRESSION SYSTEMS WITHOUT FEATURE SELECTION  WITH VALIDATION
      val = k_fold_validation(self.x_train,self.y_train,self.x_test,self.y_test)
      val.k_fold_regression()

      #PERFORMANCE OF THE REGRESSION SYSTEMS WITH FEATURE SELECTION  WITH VALIDATION
      regression_kfold(self.x_train,self.y_train,self.x_test,self.y_test,flag=None)

      print("----------------")
      print("PERFORMANCE OF THE REGRESSION SYSTEMS WITH AND WITHOUT FEATURE SELECTION WITH VALIDATION WITH REGULARISATION")

      #PERFORMANCE OF THE REGRESSION SYSTEMS WITHOUT FEATURE SELECTION WITH VALIDATION WITH REGULARISATION
      val = k_fold_validation(self.x_train,self.y_train,self.x_test,self.y_test)
      val.k_fold_regression_regularize()

      #PERFORMANCE OF THE REGRESSION SYSTEMS WITH FEATURE SELECTION  WITH VALIDATION WITH REGULARISATION
      regression_kfold(self.x_train,self.y_train,self.x_test,self.y_test)



In [ ]:
class performance_classification:

  def __init__(self,train_data_path,test_data_path,grade,mission=None):
    self.train_data_path = train_data_path
    self.test_data_path = test_data_path
    self.grade = grade
    self.mission = mission

  def data_preprocess(self):
    pre_proessing_train = pre_processing(self.train_data_path,mission = self.mission)
    pre_proessing_test = pre_processing(self.test_data_path,mission = self.mission)

    _,self.y_train = pre_proessing_train.preprocess(self.grade)
    self.x_train = pre_proessing_train.one_hot_encode()
    self.y_train_class = pre_proessing_train.create_class() 
    self.y_train_classifier = self.y_train_class

    _,self.y_test = pre_proessing_test.preprocess(self.grade)
    self.x_test = pre_proessing_test.one_hot_encode()
    self.y_test_class = pre_proessing_test.create_class() 
    self.y_test_classifier = self.y_test_class


  def reference_systems(self):

    print("PERFORMANCE OF REFERENCE SYSTEMS")
    #baseline
    baseline_class = baseline_classification(self.x_train,self.y_train_class,self.x_test,self.y_test_class)

    #trivial system
    acc_ts,macro_f1_ts,cf_ts = baseline_class.trivial_system()
    print("Performance of Trivial System")
    print("Accuracy of trivial system ", acc_ts)
    print("Macro F-1 score of trivial system ", macro_f1_ts)
    print("Confusion Matrix of trivial system \n", cf_ts)
    print("\n")

    #nearest means
    acc_nn,macro_f1_nn,cf_nn = baseline_class.nearest_means()
    print("Performance of Nearest Means system")
    print("Accuracy of Nearest Means System ", acc_nn)
    print("Macro F-1 score of Nearest Means System ", macro_f1_nn)
    print("Confusion Matrix of of  Nearest Means regressor \n", cf_nn)
    print("\n")



  def classification_systems(self,performance = None):
  
    print("PERFORMANCE OF THE CLASSIFICATION SYSTEMS WITH AND WITHOUT FEATURE SELECTION")
    #PERFORMANCE OF THE CLASSIFICATION SYSTEMS WITHOUT FEATURE SELECTION

    class_model = classification_analysis(self.x_train,self.y_train_class,self.x_test,self.y_test_class)
    class_model.perform_classification()

    #PERFORMANCE OF THE CLASSIFICATION SYSTEMS WITH FEATURE SELECTION
    classification_feature_selection(self.x_train,self.y_train_class,self.x_test,self.y_test_class,flag = None,mission = self.mission)

    print("PERFORMANCE OF THE CLASSIFICATION SYSTEMS WITH AND WITHOUT FEATURE SELECTION WITH NORMALISATION")
    #PERFORMANCE OF THE CLASSIFICATION SYSTEMS WITHOUT FEATURE SELECTION WITH NORMALISATION
    scaler = preprocessing.MinMaxScaler()
    self.x_train_normalised = self.x_train.copy()
    self.x_test_normalised  = self.x_test.copy()
    self.x_train_normalised.iloc[:]= scaler.fit_transform(self.x_train.to_numpy())
    self.x_test_normalised.iloc[:]  = scaler.transform(self.x_test.to_numpy())

    class_model = classification_analysis(self.x_train_normalised,self.y_train_class,self.x_test_normalised,self.y_test_class)
    class_model.perform_classification()

    #PERFORMANCE OF THE CLASSIFICATION SYSTEMS WITH FEATURE SELECTION WITH NORMALISATION
    classification_feature_selection(self.x_train_normalised,self.y_train_class,self.x_test_normalised,self.y_test_class,flag = None,mission = self.mission)


    if(performance == 'Normalisation'):
      
      print("PERFORMANCE OF THE CLASSIFICATION SYSTEMS WITH AND WITHOUT FEATURE SELECTION WITH NORMALISATION WITH REGULARIZATION")
      #PERFORMANCE OF THE CLASSIFICATION SYSTEMS WITHOUT FEATURE SELECTION WITH NORMALISATION WITH REGULARIZATION
      class_model = classification_analysis(self.x_train,self.y_train_class,self.x_test,self.y_test_class)
      class_model.perform_classification_regularisation()

      #PERFORMANCE OF THE CLASSIFICATION SYSTEMS WITH FEATURE SELECTION WITH NORMALISATION WITH REGULARIZATION
      classification_feature_selection(self.x_train_normalised,self.y_train_class,self.x_test_normalised,self.y_test_class,mission = self.mission)

      
      y_train_classifier = np.asarray(self.y_train_classifier)
      y_test_classifier = np.asarray(self.y_test_classifier)
      
      print("PERFORMANCE OF THE CLASSIFICATION SYSTEMS WITH AND WITHOUT FEATURE SELECTION WITH NORMALISATION WITH VALIDATION")
      #PERFORMANCE OF THE CLASSIFICATION SYSTEMS WITHOUT FEATURE SELECTION WITH NORMALISATION 
      val = k_fold_validation(self.x_train_normalised,y_train_classifier,self.x_test_normalised,y_test_classifier)
      val.k_fold_classification()

      #PERFORMANCE OF THE CLASSIFICATION SYSTEMS WITH FEATURE SELECTION WITH NORMALISATION WITH VALIDATION
      classification_kfold(self.x_train_normalised,y_train_classifier,self.x_test_normalised,y_test_classifier,flag=None)

      print("PERFORMANCE OF THE CLASSIFICATION SYSTEMS WITH AND WITHOUT FEATURE SELECTION WITH NORMALISATION WITH VALIDATION WITH REGULARISATION")
      #PERFORMANCE OF THE CLASSIFICATION SYSTEMS WITHOUT FEATURE SELECTION WITH NORMALISATION WITH VALIDATION WITH REGULARISATION
      val = k_fold_validation(self.x_train_normalised,y_train_classifier,self.x_test_normalised,y_test_classifier)
      val.k_fold_classification_regularize()

      #PERFORMANCE OF THE CLASSIFICATION SYSTEMS WITH FEATURE SELECTION WITHOUT NORMALISATION WITH VALIDATION WITH REGULARISATION
      classification_kfold(self.x_train_normalised,y_train_classifier,self.x_test_normalised,y_test_classifier)

    else:
      #PERFORMANCE OF THE CLASSIFICATION SYSTEMS WITHOUT FEATURE SELECTION WITH REGULARIZATION
      print("---------------------")
      print("PERFORMANCE OF THE CLASSIFICATION SYSTEMS WITH AND WITHOUT FEATURE SELECTION WITH REGULARIZATION")
      class_model = classification_analysis(self.x_train,self.y_train_class,self.x_test,self.y_test_class)
      class_model.perform_classification_regularisation()

      #PERFORMANCE OF THE CLASSIFICATION SYSTEMS WITH FEATURE SELECTION WITH NORMALISATION WITH REGULARIZATION
      classification_feature_selection(self.x_train,self.y_train_class,self.x_test,self.y_test_class,mission = self.mission)

      y_train_classifier = np.asarray(self.y_train_classifier)
      y_test_classifier = np.asarray(self.y_test_classifier)

      #PERFORMANCE OF THE CLASSIFICATION SYSTEMS WITHOUT FEATURE SELECTION  WITH VALIDATION
      print("---------------------")
      print("PERFORMANCE OF THE CLASSIFICATION SYSTEMS WITH AND WITHOUT FEATURE SELECTION  WITH VALIDATION")
      val = k_fold_validation(self.x_train,y_train_classifier,self.x_test,y_test_classifier)
      val.k_fold_classification()

      #PERFORMANCE OF THE CLASSIFICATION SYSTEMS WITH FEATURE SELECTION  WITH VALIDATION
      classification_kfold(self.x_train,y_train_classifier,self.x_test,self.y_test_classifier,flag=None)

      #PERFORMANCE OF THE CLASSIFICATION SYSTEMS WITHOUT FEATURE SELECTION WITH VALIDATION WITH REGULARISATION
      print("---------------------")
      print("PERFORMANCE OF THE CLASSIFICATION SYSTEMS WITH AND WITHOUT FEATURE SELECTION WITH VALIDATION WITH REGULARISATION")
      val = k_fold_validation(self.x_train,y_train_classifier,self.x_test,y_test_classifier)
      val.k_fold_classification_regularize()

      #PERFORMANCE OF THE CLASSIFICATION SYSTEMS WITH FEATURE SELECTION  WITH VALIDATION WITH REGULARISATION
      classification_kfold(self.x_train,y_train_classifier,self.x_test,y_test_classifier)

MISSION-1 REGRESSION

In [ ]:
performance_reg = performance_regression('student_performance_train.csv','student_performance_test.csv','G1')
performance_reg.data_preprocess()
performance_reg.reference_systems()
print("----------------------------------------------------------------")
print("The performance of regression systems")
performance_reg.regression_systems(performance = 'Normalisation')
print("----------------------------------------------------------------")

MISSION-1 CLASSIFICATION

In [ ]:
performance_class = performance_classification('student_performance_train.csv','student_performance_test.csv','G1')
performance_class.data_preprocess()
performance_class.reference_systems()
print("----------------------------------------------------------------")
print("The performance of classification systems")
performance_class.classification_systems()
print("----------------------------------------------------------------")

MISSION-2 REGRESSION

In [ ]:
performance_reg = performance_regression('student_performance_train.csv','student_performance_test.csv','G3')
performance_reg.data_preprocess()
performance_reg.reference_systems()
print("----------------------------------------------------------------")
print("The performance of regression systems")
performance_reg.regression_systems()
print("----------------------------------------------------------------")

MISSION-2 CLASSIFICATION

In [ ]:
performance_class = performance_classification('student_performance_train.csv','student_performance_test.csv','G3')
performance_class.data_preprocess()
print("The performance of reference systems")
performance_class.reference_systems()
print("----------------------------------------------------------------")
print("The performance of regression systems")
performance_class.classification_systems(performance = 'Normalisation')
print("----------------------------------------------------------------")

MISSION-3 REGRESSION

In [ ]:
performance_reg = performance_regression('student_performance_train.csv','student_performance_test.csv','G3',mission = '3')
performance_reg.data_preprocess()
performance_reg.reference_systems()
print("----------------------------------------------------------------")
print("The performance of regression systems")
performance_reg.regression_systems()
print("----------------------------------------------------------------")

MISSION-3 CLASSIFICATION

In [ ]:
performance_class = performance_classification('student_performance_train.csv','student_performance_test.csv','G3',mission = '3')
performance_class.data_preprocess()
print("The performance of reference systems")
performance_class.reference_systems()
print("----------------------------------------------------------------")
print("The performance of regression systems")
performance_class.classification_systems()
print("----------------------------------------------------------------")

Hyper parameter tuning for mission 1

In [ ]:
pre_proessing_train_m1r = pre_processing('student_performance_train.csv')
pre_proessing_test_m1r = pre_processing('student_performance_test.csv')

_,y_train_m1r = pre_proessing_train_m1r.preprocess('G1')
x_train_m1r = pre_proessing_train_m1r.one_hot_encode()
y_train_class_m1r = pre_proessing_train_m1r.create_class() 
y_train_classifier_m1r = y_train_class_m1r

_,y_test_m1r = pre_proessing_test_m1r.preprocess('G1')
x_test_m1r = pre_proessing_test_m1r.one_hot_encode()
y_test_class_m1r = pre_proessing_test_m1r.create_class() 
y_test_classifier_m1r = y_test_class_m1r

scaler = preprocessing.MinMaxScaler()
x_train_normalised_m1r = x_train_m1r.copy()
x_test_normalised_m1r  = x_test_m1r.copy()
x_train_normalised_m1r.iloc[:]= scaler.fit_transform(x_train_m1r.to_numpy())
x_test_normalised_m1r.iloc[:]  = scaler.transform(x_test_m1r.to_numpy())

In [ ]:
#mission1 regression - gradient boost + RFE 

rmse_list = []
r2_list = []
for i in range(1,51):
  rfe_selector = RFE(estimator=LinearRegression(),n_features_to_select = i, step = 2)
  rfe_selector.fit(x_train_normalised_m1r,y_train_m1r)
  col_req_list = x_train_normalised_m1r.columns[rfe_selector.get_support()]

  reg = ensemble.GradientBoostingRegressor().fit(x_train_normalised_m1r[col_req_list], y_train_m1r)
  y_pred = reg.predict(x_test_normalised_m1r[col_req_list])
  mse = mean_squared_error(y_test_m1r, y_pred)
  rmse =  mean_squared_error(y_test_m1r, y_pred, squared=False)
  rmse_list.append(rmse)
  r2 = r2_score(y_test_m1r, y_pred) 
  r2_list.append(r2)

plt.figure(0)
plt.plot(rmse_list,label = 'rmse list')
plt.plot(r2_list, label = 'r2 list')
plt.legend()
min_rmse_index_rfe = rmse_list.index(min(rmse_list)) 
max_r2_index_rfe = r2_list.index(max(r2_list)) 


rfe_selector = RFE(estimator=LinearRegression(),n_features_to_select = min_rmse_index_rfe, step = 2)
rfe_selector.fit(x_train_normalised_m1r,y_train_m1r)
col_req_list = x_train_normalised_m1r.columns[rfe_selector.get_support()]
rmse_list = []
r2_list = []

for i in range(1,201):
  reg = ensemble.GradientBoostingRegressor(n_estimators = i).fit(x_train_normalised_m1r[col_req_list], y_train_m1r)
  y_pred = reg.predict(x_test_normalised_m1r[col_req_list])
  mse = mean_squared_error(y_test_m1r, y_pred)
  rmse =  mean_squared_error(y_test_m1r, y_pred, squared=False)
  rmse_list.append(rmse)
  r2 = r2_score(y_test_m1r, y_pred) 
  r2_list.append(r2)

plt.figure(1)
plt.plot(rmse_list,label = 'rmse list')
plt.plot(r2_list, label = 'r2 list')
plt.legend()
min_rmse_index_gb = rmse_list.index(min(rmse_list)) 
max_r2_index_gb = r2_list.index(max(r2_list)) 

rfe_selector = RFE(estimator=LinearRegression(),n_features_to_select = min_rmse_index_rfe, step = 2)
rfe_selector.fit(x_train_normalised_m1r,y_train_m1r)
col_req_list = x_train_normalised_m1r.columns[rfe_selector.get_support()]


reg = ensemble.GradientBoostingRegressor(n_estimators = min_rmse_index_gb).fit(x_train_normalised_m1r[col_req_list], y_train_m1r)
y_pred = reg.predict(x_test_normalised_m1r[col_req_list])
mse = mean_squared_error(y_test_m1r, y_pred)
rmse =  mean_squared_error(y_test_m1r, y_pred, squared=False)
r2 = r2_score(y_test_m1r, y_pred) 

print(rmse)
print(r2)


In [ ]:
pre_proessing_train_m2r = pre_processing('student_performance_train.csv')
pre_proessing_test_m2r = pre_processing('student_performance_test.csv')

_,y_train_m2r = pre_proessing_train_m2r.preprocess('G3')
x_train_m2r = pre_proessing_train_m2r.one_hot_encode()
y_train_class_m2r = pre_proessing_train_m2r.create_class() 
y_train_classifier_m2r = y_train_class_m2r

_,y_test_m2r = pre_proessing_test_m2r.preprocess('G3')
x_test_m2r = pre_proessing_test_m2r.one_hot_encode()
y_test_class_m2r = pre_proessing_test_m2r.create_class() 
y_test_classifier_m2r = y_test_class_m2r

scaler_m2r = preprocessing.MinMaxScaler()
x_train_normalised_m2r = x_train_m2r.copy()
x_test_normalised_m2r  = x_test_m2r.copy()
x_train_normalised_m2r.iloc[:]= scaler_m2r.fit_transform(x_train_m2r.to_numpy())
x_test_normalised_m2r.iloc[:]  = scaler_m2r.transform(x_test_m2r.to_numpy())

In [ ]:
#mission 2 regression

rmse_list = []
r2_list = []
for i in range(1,26):
  selector = SelectKBest(chi2, k = i)
  selector.fit(x_train_m2r, y_train_m2r)
  selected_feat = x_train_m2r.columns[selector.get_support()]

  reg = ensemble.GradientBoostingRegressor().fit(x_train_m2r[selected_feat], y_train_m2r)
  y_pred = reg.predict(x_test_m2r[selected_feat])
  mse = mean_squared_error(y_test_m2r, y_pred)
  rmse =  mean_squared_error(y_test_m2r, y_pred, squared=False)
  rmse_list.append(rmse)
  r2 = r2_score(y_test_m2r, y_pred) 
  r2_list.append(r2)

plt.figure(0)
plt.plot(rmse_list,label = 'rmse list')
plt.plot(r2_list, label = 'r2 list')
plt.legend()
min_rmse_index_rfe = rmse_list.index(min(rmse_list)) 
max_r2_index_rfe = r2_list.index(max(r2_list)) 

print(min_rmse_index_rfe)
print(max_r2_index_rfe)


selector = SelectKBest(chi2, k = min_rmse_index_rfe)
selector.fit(x_train_m2r, y_train_m2r)
selected_feat = x_train_m2r.columns[selector.get_support()]
rmse_list = []
r2_list = []

for i in range(1,201):
  reg = ensemble.GradientBoostingRegressor(n_estimators = i).fit(x_train_m2r[selected_feat], y_train_m2r)
  y_pred = reg.predict(x_test_m2r[selected_feat])
  mse = mean_squared_error(y_test_m2r, y_pred)
  rmse =  mean_squared_error(y_test_m2r, y_pred, squared=False)
  rmse_list.append(rmse)
  r2 = r2_score(y_test_m2r, y_pred) 
  r2_list.append(r2)

plt.figure(1)
plt.plot(rmse_list,label = 'rmse list')
plt.plot(r2_list, label = 'r2 list')
plt.legend()
min_rmse_index_gb = rmse_list.index(min(rmse_list)) 
max_r2_index_gb = r2_list.index(max(r2_list)) 

print(min_rmse_index_gb)
print(max_r2_index_gb)


selector = SelectKBest(chi2, k = min_rmse_index_rfe)
selector.fit(x_train_m2r, y_train_m2r)
selected_feat = x_train_m2r.columns[selector.get_support()]


reg = ensemble.GradientBoostingRegressor().fit(x_train_m2r[selected_feat], y_train_m2r)
y_pred = reg.predict(x_test_m2r[selected_feat])
mse = mean_squared_error(y_test_m2r, y_pred)
rmse =  mean_squared_error(y_test_m2r, y_pred, squared=False)
r2 = r2_score(y_test_m2r, y_pred) 

print(rmse)
print(r2)


In [ ]:
proessing_train = pre_processing('student_performance_train.csv',mission='3')
pre_proessing_test = pre_processing('student_performance_test.csv',mission='3')

_,y_train = proessing_train.preprocess('G3')
x_train = proessing_train.one_hot_encode()
y_train_class = proessing_train.create_class() 
y_train_classifier = y_train_class

_,y_test = pre_proessing_test.preprocess('G3')
x_test = pre_proessing_test.one_hot_encode()
y_test_class = pre_proessing_test.create_class() 
y_test_classifier = y_test_class

In [ ]:
#mission3 regression - ANN + mutual info


rmse_list = []
r2_list = []
for i in range(1,26):
  selector = SelectKBest(mutual_info_regression, k = i)
  selector.fit(x_train, y_train)
  selected_feat = x_train.columns[selector.get_support()]

  regr = MLPRegressor(random_state=1, max_iter=500).fit(x_train[selected_feat], y_train)
  y_pred = regr.predict(x_test[selected_feat])
  mse = mean_squared_error(y_test, y_pred)
  rmse =  mean_squared_error(y_test, y_pred, squared=False)
  r2 = r2_score(y_test, y_pred)   
  rmse_list.append(rmse)
  r2 = r2_score(y_test, y_pred) 
  r2_list.append(r2)

plt.figure(0)
plt.plot(rmse_list,label = 'rmse list')
plt.plot(r2_list, label = 'r2 list')
plt.legend()
min_rmse_index_rfe = rmse_list.index(min(rmse_list)) 
max_r2_index_rfe = r2_list.index(max(r2_list)) 

print(min_rmse_index_rfe)
print(max_r2_index_rfe)
print(min(rmse_list))
print(max(r2_list))

PLOTTING GRAPH

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

fs_list = ['all_feat','random_for ','mutual_info ','chi_squared ','seq_feature','RFE']

#SVR
 #-w/o norm, with norm , with reg+norm,
 # -w/o norm, with norm and val, with reg+norm, with reg+val+norm

data_svr = [[2.46,2.64,2.43,2.40,2.42,2.48],
[2.45,2.64,2.55,2.55,2.42,2.46],
[2.33,2.52,2.36,2.29,2.27,2.27]]


data_ann = [[2.50,2.74,2.47,2.46,2.37,2.38],
[2.64,2.64,2.53,2.53,2.37,2.40],
[2.47,2.57,2.35,2.35,2.28,2.31]]


data_gb = [[2.30,2.74,2.45,2.35,2.38,2.37],
[2.31,2.74,2.44,2.36,2.38,2.35],
[2.37,2.54,2.42,2.35,2.32,2.36]]

data_knn = [[2.70,2.72,2.61,2.64,2.57,2.63],
[2.63,2.46,2.57,2.57,2.55,2.55],
[2.42,2.51,2.40,2.41,2.32,2.33]]

X = np.arange(6)
fig, axs = plt.subplots(2, 2,figsize=(16,16))
fig.suptitle('MISSION-I Regression',size=16)
plt.setp(axs,xticks = range(len(fs_list)), xticklabels= fs_list)

for ax in axs.flat:
    ax.set( ylabel='RMSE')
    ax.tick_params(axis='x', labelrotation=45)


axs[0,0].bar(X + 0.00, data_svr[0], color = 'c', width = 0.25, label = 'without norm')
axs[0,0].bar(X + 0.25, data_svr[1], color = '#497ce3', width = 0.25,label = 'with norm' )
axs[0,0].bar(X + 0.50, data_svr[2], color = '#8649e3', width = 0.25, label = 'with norm and cross-validation')
axs[0,0].legend(loc="upper right",prop={'size': 7})
axs[0,0].set_title('SVR')

axs[0,1].bar(X + 0.00, data_ann[0], color = 'c', width = 0.25, label = 'without norm')
axs[0,1].bar(X + 0.25, data_ann[1], color = '#497ce3', width = 0.25,label = 'with norm' )
axs[0,1].bar(X + 0.50, data_ann[2], color = '#8649e3', width = 0.25, label = 'with norm and cross-validation')
axs[0,1].legend(loc="upper right",prop={'size': 7})
axs[0,1].set_title('ANN')

axs[1,0].bar(X + 0.00, data_gb[0], color = 'c', width = 0.25, label = 'without norm')
axs[1,0].bar(X + 0.25, data_gb[1], color = '#497ce3', width = 0.25,label = 'with norm' )
axs[1,0].bar(X + 0.50, data_gb[2], color = '#8649e3', width = 0.25, label = 'with norm and cross-validation')
axs[1,0].legend(loc="upper right",prop={'size': 7})
axs[1,0].set_title('Gradient Boost')

axs[1,1].bar(X + 0.00, data_knn[0], color = 'c', width = 0.25, label = 'without norm')
axs[1,1].bar(X + 0.25, data_knn[1], color = '#497ce3', width = 0.25,label = 'with norm' )
axs[1,1].bar(X + 0.50, data_knn[2], color = '#8649e3', width = 0.25, label = 'with norm and cross-validation')
axs[1,1].legend(loc="upper right",prop={'size': 7})
axs[1,1].set_title('KNN')

In [ ]:
fs_list = ['all_features','random forest','mutual info','chi_squared','seq_feature','RFE']

#SVR
 #-w/o norm, with norm , with reg+norm,
 # -w/o norm, with norm and val, with reg+norm, with reg+val+norm

data_svr = [[2.45,2.63,2.50,2.50,2.42,2.46],
[2.35,2.53,2.30,2.30,2.28,2.28]]

data_ann = [[2.58,2.63,2.46,2.46,2.37,2.39],
[2.47,2.57,2.31,2.35,2.28,2.33]]

data_ridge = [[2.42,2.63,2.45,2.45,2.40,2.42],
[2.32,2.52,2.27,2.27,2.26,2.26]]

data_lasso = [[2.55,2.73,2.55,2.55,2.55,2.55],
[2.37,2.57,2.37,2.37,2.37,2.3]]


X = np.arange(6)
fig, axs = plt.subplots(2, 2,figsize=(15,15))
fig.suptitle('MISSION-I Regression')
plt.setp(axs,xticks = range(len(fs_list)), xticklabels= fs_list)

for ax in axs.flat:
    ax.set( ylabel='RMSE')
    ax.tick_params(axis='x', labelrotation=45)

axs[0,0].bar(X + 0.00, data_svr[0], color = 'c', width = 0.25, label = 'with regularization')
axs[0,0].bar(X + 0.25, data_svr[1], color = '#497ce3', width = 0.25,label = 'with regularization and cross-validation' )
axs[0,0].legend(loc="upper right",prop={'size': 7})
axs[0,0].set_title('SVR')

axs[0,1].bar(X + 0.00, data_ann[0], color = 'c', width = 0.25, label = 'with regularization')
axs[0,1].bar(X + 0.25, data_ann[1], color = '#497ce3', width = 0.25,label = 'with regularization and cross-validation' )
axs[0,1].legend(loc="upper right",prop={'size': 7})
axs[0,1].set_title('ANN')

axs[1,0].bar(X + 0.00, data_ridge[0], color = 'c', width = 0.25, label = 'with regularization')
axs[1,0].bar(X + 0.25, data_ridge[1], color = '#497ce3', width = 0.25,label = 'with regularization and cross-validation' )
axs[1,0].legend(loc="upper right",prop={'size': 7})
axs[1,0].set_title('Ridge')

axs[1,1].bar(X + 0.00, data_lasso[0], color = 'c', width = 0.25, label = 'with regularization')
axs[1,1].bar(X + 0.25, data_lasso[1], color = '#497ce3', width = 0.25,label = 'with regularization and cross-validation')
axs[1,1].legend(loc="upper right",prop={'size': 7})
axs[1,1].set_title('Lasso')

In [ ]:
#mission 1 - classifcation

data_lr = [[0.30,0.32,0.27,0.30,0.30,0.34],
[0.30,0.32,0.24,0.30,0.30,0.34],
[0.27,0.27,0.26,0.27,0.32,0.39]]

data_svm = [[0.28,0.27,0.27,0.31,0.32,0.32],
[0.28,0.27,0.27,0.31,0.32,0.32],
[0.22,0.26,0.27,0.25,0.30,0.30]]

data_gau = [[0.18,0.30,0.18,0.17,0.26,0.27],
[0.19,0.31,0.21,0.17,0.26,0.27],
[0.17,0.08,0.19,0.08,0.20,0.20]]

data_rf = [[0.22,0.22,0.27,0.24,0.23,0.21],
[0.22,0.22,0.25,0.24,0.23,0.22],
[0.24,0.22,0.22,0.24,0.24,0.22]]

data_mlp = [[0.29,0.32,0.30,0.31,0.35,0.34],
[0.29,0.32,0.29,0.31,0.35,0.34],
[0.27,0.29,0.26,0.27,0.35,0.30]]

data_nb = [[0.21,0.17,0.16,0.21,0.22,0.20],
[0.21,0.18,0.20,0.21,0.22,0.21],
[0.19,0.16,0.17,0.19,0.20,0.18]]

data_knn = [[0.24,0.25,0.28,0.25,0.30,0.28],
[0.24,0.25,0.23,0.25,0.32,0.28],
[0.22,0.21,0.27,0.23,0.28,0.25]]

X = np.arange(6)
fig, axs = plt.subplots(4, 2,figsize=(19,19))
fig.suptitle('MISSION-I Classification')
plt.setp(axs,xticks = range(len(fs_list)), xticklabels= fs_list)
axs[3,1].set_axis_off()
for ax in axs.flat:
    ax.set( ylabel='Macro F-1 score')
    #ax.tick_params(axis='x', labelrotation=30)

axs[0,0].bar(X + 0.00, data_lr[0], color = 'c', width = 0.25, label = 'without norm')
axs[0,0].bar(X + 0.25, data_lr[1], color = '#497ce3', width = 0.25,label = 'with norm' )
axs[0,0].bar(X + 0.50, data_lr[2], color = '#8649e3', width = 0.25, label = 'without norm and cross-validation')
axs[0,0].legend(loc="upper right",prop={'size': 7})
axs[0,0].set_title('Logistic Regression')

axs[0,1].bar(X + 0.00, data_svm[0], color = 'c', width = 0.25, label = 'without norm')
axs[0,1].bar(X + 0.25, data_svm[1], color = '#497ce3', width = 0.25,label = 'with norm' )
axs[0,1].bar(X + 0.50, data_svm[2], color = '#8649e3', width = 0.25, label = 'without norm and cross-validation')
axs[0,1].legend(loc="upper right",prop={'size': 7})
axs[0,1].set_title('SVM')

axs[1,0].bar(X + 0.00, data_gau[0], color = 'c', width = 0.25, label = 'without norm')
axs[1,0].bar(X + 0.25, data_gau[1], color = '#497ce3', width = 0.25,label = 'with norm' )
axs[1,0].bar(X + 0.50, data_gau[2], color = '#8649e3', width = 0.25, label = 'without norm and cross-validation')
axs[1,0].legend(loc="upper right",prop={'size': 7})
axs[1,0].set_title('Gaussian')

axs[1,1].bar(X + 0.00, data_rf[0], color = 'c', width = 0.25, label = 'without norm')
axs[1,1].bar(X + 0.25, data_rf[1], color = '#497ce3', width = 0.25,label = 'with norm' )
axs[1,1].bar(X + 0.50, data_rf[2], color = '#8649e3', width = 0.25, label = 'without norm and cross-validation')
axs[1,1].legend(loc="upper right",prop={'size': 7})
axs[1,1].set_title('Random Forest')

axs[2,0].bar(X + 0.00, data_mlp[0], color = 'c', width = 0.25, label = 'without norm')
axs[2,0].bar(X + 0.25, data_mlp[1], color = '#497ce3', width = 0.25,label = 'with norm' )
axs[2,0].bar(X + 0.50, data_mlp[2], color = '#8649e3', width = 0.25, label = 'without norm and cross-validation')
axs[2,0].legend(loc="upper right",prop={'size': 7})
axs[2,0].set_title('MLP')

axs[2,1].bar(X + 0.00, data_nb[0], color = 'c', width = 0.25, label = 'without norm')
axs[2,1].bar(X + 0.25, data_nb[1], color = '#497ce3', width = 0.25,label = 'with norm' )
axs[2,1].bar(X + 0.50, data_nb[2], color = '#8649e3', width = 0.25, label = 'without norm and cross-validation')
axs[2,1].legend(loc="upper right",prop={'size': 7})
axs[2,1].set_title('Naive bayes')

axs[3,0].bar(X + 0.00, data_knn[0], color = 'c', width = 0.25, label = 'without norm')
axs[3,0].bar(X + 0.25, data_knn[1], color = '#497ce3', width = 0.25,label = 'with norm' )
axs[3,0].bar(X + 0.50, data_knn[2], color = '#8649e3', width = 0.25, label = 'without norm and cross-validation')
axs[3,0].legend(loc="upper right",prop={'size': 7})
axs[3,0].set_title('KNN')

In [ ]:
#reg and val

#logistic l1 
data_l1 = [[0.30,0.31,0.32,0.33,0.32,0.31],
[0.28,0.26,0.25,0.28,0.30,0.38]]

data_l2 = [[0.29,0.30,0.30,0.31,0.32,0.31],
[0.27,0.26,0.25,0.27,0.32,0.30]]

data_svm = [[0.29,0.29,0.30,0.30,0.37,0.31,0.30],
[0.27,0.28,0.27,0.26,0.30,0.32,0]]

data_mlp = [[0.27,0.26,0.30,0.30,0.37,0.30,0.30],
[0.26,0.25,0.26,0.28,0.32,0.30,0]]

X = np.arange(6)
fig, axs = plt.subplots(2, 2,figsize=(15,15))
fig.suptitle('MISSION-I Classification')
plt.setp(axs,xticks = range(len(fs_list)), xticklabels= fs_list)

for ax in axs.flat:
    ax.set( ylabel='Macro F-1 score')
    ax.tick_params(axis='x', labelrotation=45)    

axs[0,0].bar(X + 0.00, data_svr[0], color = 'c', width = 0.25, label = 'with regularization')
axs[0,0].bar(X + 0.25, data_svr[1], color = '#497ce3', width = 0.25,label = 'with regularization and cross-validation' )
axs[0,0].legend(loc="upper right",prop={'size': 7})
axs[0,0].set_title('Logistic regression L1')

axs[0,1].bar(X + 0.00, data_ann[0], color = 'c', width = 0.25, label = 'with regularization')
axs[0,1].bar(X + 0.25, data_ann[1], color = '#497ce3', width = 0.25,label = 'with regularization and cross-validation' )
axs[0,1].legend(loc="upper right",prop={'size': 7})
axs[0,1].set_title('Logistic regression L2')

axs[1,0].bar(X + 0.00, data_ridge[0], color = 'c', width = 0.25, label = 'with regularization')
axs[1,0].bar(X + 0.25, data_ridge[1], color = '#497ce3', width = 0.25,label = 'with regularization and cross-validation' )
axs[1,0].legend(loc="upper right",prop={'size': 7})
axs[1,0].set_title('SVM')

axs[1,1].bar(X + 0.00, data_lasso[0], color = 'c', width = 0.25, label = 'with regularization')
axs[1,1].bar(X + 0.25, data_lasso[1], color = '#497ce3', width = 0.25,label = 'with regularization and cross-validation')
axs[1,1].legend(loc="upper right",prop={'size': 7})
axs[1,1].set_title('MLP')


In [ ]:
#mission 2 

#SVM -w/o norm, with norm,val
data_svr = [[2.72,2.96,2.80,2.73,2.81,2.78],
[2.70,2.96,2.74,2.74,2.80,2.72],
[2.78,3.00,2.80,2.73,2.71,2.76]]

data_ann = [[2.83,3.09,2.80,2.80,2.76,2.67],
[2.85,3.00,2.69,2.70,2.86,2.71],
[2.80,3.10,2.81,2.80,2.73,2.78]]

data_gb = [[2.64,3.15,2.84,2.62,2.82,2.83],
[2.64,3.15,2.78,2.68,2.82,2.87],
[2.87,3.04,2.92,2.86,2.77,2.84]]

data_knn = [[3,3.04,3.01,2.97,2.97,2.73],
[2.89,3.03,2.81,2.77,2.86,2.84],
[2.91,2.98,3.01,2.97,2.95,2.85]]


X = np.arange(6)
fig, axs = plt.subplots(2, 2,figsize=(15,15))
fig.suptitle('MISSION-II Regression')
plt.setp(axs,xticks = range(len(fs_list)), xticklabels= fs_list)

for ax in axs.flat:
    ax.set( ylabel='RMSE')
    ax.tick_params(axis='x', labelrotation=45)

axs[0,0].bar(X + 0.00, data_svr[0], color = 'c', width = 0.25, label = 'without norm')
axs[0,0].bar(X + 0.25, data_svr[1], color = '#497ce3', width = 0.25,label = 'with norm' )
axs[0,0].bar(X + 0.50, data_svr[2], color = '#8649e3', width = 0.25, label = 'without norm and cross-validation')
axs[0,0].legend(loc="upper right",prop={'size': 7})
axs[0,0].set_title('SVR')

axs[0,1].bar(X + 0.00, data_ann[0], color = 'c', width = 0.25, label = 'without norm')
axs[0,1].bar(X + 0.25, data_ann[1], color = '#497ce3', width = 0.25,label = 'with norm' )
axs[0,1].bar(X + 0.50, data_ann[2], color = '#8649e3', width = 0.25, label = 'without norm and cross-validation')
axs[0,1].legend(loc="upper right",prop={'size': 7})
axs[0,1].set_title('ANN')

axs[1,0].bar(X + 0.00, data_gb[0], color = 'c', width = 0.25, label = 'without norm')
axs[1,0].bar(X + 0.25, data_gb[1], color = '#497ce3', width = 0.25,label = 'with norm' )
axs[1,0].bar(X + 0.50, data_gb[2], color = '#8649e3', width = 0.25, label = 'without norm and cross-validation')
axs[1,0].legend(loc="upper right",prop={'size': 7})
axs[1,0].set_title('Gradient Boost')

axs[1,1].bar(X + 0.00, data_knn[0], color = 'c', width = 0.25, label = 'without norm')
axs[1,1].bar(X + 0.25, data_knn[1], color = '#497ce3', width = 0.25,label = 'with norm' )
axs[1,1].bar(X + 0.50, data_knn[2], color = '#8649e3', width = 0.25, label = 'without norm and cross-validation')
axs[1,1].legend(loc="upper right",prop={'size': 7})
axs[1,1].set_title('KNN')


In [ ]:
#reg and val

#svr
data_svr = [[2.71,2.96,2.77,2.77,2.81,2.70],
[2.78,3.00,2.82,2.73,2.71,2.76]]

data_ann = [[2.90,3.00,2.75,2.76,2.78,2.71],
[2.81,3.08,2.77,2.79,2.73,2.76]]

data_ridge = [[2.71,3.00,2.75,2.75,2.79,2.70],
[2.78,3.02,2.80,2.74,2.69,2.74]]

data_lasso = [[2.79,3.05,2.80,2.80,2.80,2.80],
[2.78,3.02,2.81,2.77,2.75,2.76]]


X = np.arange(6)
fig, axs = plt.subplots(2, 2,figsize=(15,15))
fig.suptitle('MISSION-II Regression')
plt.setp(axs,xticks = range(len(fs_list)), xticklabels= fs_list)

for ax in axs.flat:
    ax.set( ylabel='RMSE')
    ax.tick_params(axis='x', labelrotation=45)

axs[0,0].bar(X + 0.00, data_svr[0], color = 'c', width = 0.25, label = 'with regularization')
axs[0,0].bar(X + 0.25, data_svr[1], color = '#497ce3', width = 0.25,label = 'with regularization and cross-validation' )
axs[0,0].legend(loc="upper right",prop={'size': 7})
axs[0,0].set_title('SVR')

axs[0,1].bar(X + 0.00, data_ann[0], color = 'c', width = 0.25, label = 'with regularization')
axs[0,1].bar(X + 0.25, data_ann[1], color = '#497ce3', width = 0.25,label = 'with regularization and cross-validation' )
axs[0,1].legend(loc="upper right",prop={'size': 7})
axs[0,1].set_title('ANN')

axs[1,0].bar(X + 0.00, data_ridge[0], color = 'c', width = 0.25, label = 'with regularization')
axs[1,0].bar(X + 0.25, data_ridge[1], color = '#497ce3', width = 0.25,label = 'with regularization and cross-validation' )
axs[1,0].legend(loc="upper right",prop={'size': 7})
axs[1,0].set_title('Ridge')

axs[1,1].bar(X + 0.00, data_lasso[0], color = 'c', width = 0.25, label = 'with regularization')
axs[1,1].bar(X + 0.25, data_lasso[1], color = '#497ce3', width = 0.25,label = 'with regularization and cross-validation')
axs[1,1].legend(loc="upper right",prop={'size': 7})
axs[1,1].set_title('Lasso')

In [ ]:
#logisitc reg -w/o norm, with norm, with validation 
data_lr = [[0.31,0.18,0.28,0.30,0.26,0.26],
[0.30,0.19,0.22,0.32,0.28,0.26],
[0.30,0.27,0.29,0.31,0.33,0.33]]

data_svm = [[0.31,0.09,0.28,0.28,0.20,0.26],
[0.25,0.25,0.23,0.25,0.27,0.28],
[0.27,0.26,0.30,0.31,0.32,0.35]]

data_gau = [[0.14,0.09,0.20,0.09,0.18,0.20],
[0.21,0.20,0.23,0.24,0.27,0.26],
[0.26,0.25,0.21,0.27,0.29,0.31]]

data_rf = [[0.16,0.13,0.16,0.16,0.19,0.16],
[0.16,0.14,0.17,0.16,0.19,0.15],
[0.20,0.26,0.23,0.22,0.29,0.27]]

data_mlp = [[0.31,0.25,0.30,0.32,0.29,0.30],
[0.37,0.27,0.32,0.31,0.27,0.30],
[0.15,0.16,0.21,0.16,0.22,0.17]]

data_nb = [[0.27,0.20,0.29,0.26,0.27,0.21],
[0.25,0.17,0.25,0.22,0.24,0.27],
[0.32,0.32,0.32,0.34,0.34,0.36]]

data_knn = [[0.20,0.17,0.18,0.18,0.16,0.19],
[0.22,0.27,0.22,0.25,0.29,0.17],
[0.25,0.20,0.23,0.25,0.23,0.26]]

X = np.arange(6)
fig, axs = plt.subplots(4, 2,figsize=(17,17))
fig.suptitle('MISSION-II Classification')
plt.setp(axs,xticks = range(len(fs_list)), xticklabels= fs_list)
axs[3,1].set_axis_off()
for ax in axs.flat:
    ax.set( ylabel='Macro F-1 Score')


axs[0,0].bar(X + 0.00, data_lr[0], color = 'c', width = 0.25, label = 'without norm')
axs[0,0].bar(X + 0.25, data_lr[1], color = '#497ce3', width = 0.25,label = 'with norm' )
axs[0,0].bar(X + 0.50, data_lr[2], color = '#8649e3', width = 0.25, label = 'with norm and cross-validation')
axs[0,0].legend(loc="upper right",prop={'size': 7})
axs[0,0].set_title('Logistic Regression')

axs[0,1].bar(X + 0.00, data_svm[0], color = 'c', width = 0.25, label = 'without norm')
axs[0,1].bar(X + 0.25, data_svm[1], color = '#497ce3', width = 0.25,label = 'with norm' )
axs[0,1].bar(X + 0.50, data_svm[2], color = '#8649e3', width = 0.25, label = 'with norm and cross-validation')
axs[0,1].legend(loc="upper right",prop={'size': 7})
axs[0,1].set_title('SVM')

axs[1,0].bar(X + 0.00, data_gau[0], color = 'c', width = 0.25, label = 'without norm')
axs[1,0].bar(X + 0.25, data_gau[1], color = '#497ce3', width = 0.25,label = 'with norm' )
axs[1,0].bar(X + 0.50, data_gau[2], color = '#8649e3', width = 0.25, label = 'with norm and cross-validation')
axs[1,0].legend(loc="upper right",prop={'size': 7})
axs[1,0].set_title('Gaussian')

axs[1,1].bar(X + 0.00, data_rf[0], color = 'c', width = 0.25, label = 'without norm')
axs[1,1].bar(X + 0.25, data_rf[1], color = '#497ce3', width = 0.25,label = 'with norm' )
axs[1,1].bar(X + 0.50, data_rf[2], color = '#8649e3', width = 0.25, label = 'with norm and cross-validation')
axs[1,1].legend(loc="upper right",prop={'size': 7})
axs[1,1].set_title('Random Forest')

axs[2,0].bar(X + 0.00, data_mlp[0], color = 'c', width = 0.25, label = 'without norm')
axs[2,0].bar(X + 0.25, data_mlp[1], color = '#497ce3', width = 0.25,label = 'with norm' )
axs[2,0].bar(X + 0.50, data_mlp[2], color = '#8649e3', width = 0.25, label = 'with norm and cross-validation')
axs[2,0].legend(loc="upper right",prop={'size': 7})
axs[2,0].set_title('MLP')

axs[2,1].bar(X + 0.00, data_nb[0], color = 'c', width = 0.25, label = 'without norm')
axs[2,1].bar(X + 0.25, data_nb[1], color = '#497ce3', width = 0.25,label = 'with norm' )
axs[2,1].bar(X + 0.50, data_nb[2], color = '#8649e3', width = 0.25, label = 'with norm and cross-validation')
axs[2,1].legend(loc="upper right",prop={'size': 7})
axs[2,1].set_title('Naive Bayes')

axs[3,0].bar(X + 0.00, data_knn[0], color = 'c', width = 0.25, label = 'without norm')
axs[3,0].bar(X + 0.25, data_knn[1], color = '#497ce3', width = 0.25,label = 'with norm' )
axs[3,0].bar(X + 0.50, data_knn[2], color = '#8649e3', width = 0.25, label = 'with norm and cross-validation')
axs[3,0].legend(loc="upper right",prop={'size': 7})
axs[3,0].set_title('KNN')

In [ ]:
#reg and val

#logisitic reg - l1
data_l1 = [[0.25,0.21,0.18,0.27,0.24,0.26],
[0.31,0.22,0.24,0.28,0.33,0.32]]

data_l2 = [[0.31,0.19,0.21,0.32,0.28,0.26],
[0.30,0.23,0.25,0.31,0.33,0.33]]

data_svm = [[0.29,0.20,0.28,0.34,0.29,0.27],
[0.30,0.28,0.32,0.31,0.38,0.39]]

data_mlp = [[0.30,0.25,0.31,0.33,0.28,0.31],
[0.33,0.29,0.28,0.33,0.34,0.36]]



X = np.arange(6)
fig, axs = plt.subplots(2, 2,figsize=(15,15))
fig.suptitle('MISSION-II Classification')
plt.setp(axs,xticks = range(len(fs_list)), xticklabels= fs_list)

for ax in axs.flat:
    ax.set( ylabel='Macro F-1 score')
    ax.tick_params(axis='x', labelrotation=45)

axs[0,0].bar(X + 0.00, data_l1[0], color = 'c', width = 0.25, label = 'with regularization')
axs[0,0].bar(X + 0.25, data_l1[1], color = '#497ce3', width = 0.25,label = 'with regularization and cross-validation' )
axs[0,0].legend(loc="upper right",prop={'size': 7})
axs[0,0].set_title('Logisitc Regression L1')

axs[0,1].bar(X + 0.00, data_l2[0], color = 'c', width = 0.25, label = 'with regularization')
axs[0,1].bar(X + 0.25, data_l2[1], color = '#497ce3', width = 0.25,label = 'with regularization and cross-validation' )
axs[0,1].legend(loc="upper right",prop={'size': 7})
axs[0,1].set_title('Logisitc Regression L2')

axs[1,0].bar(X + 0.00, data_svm[0], color = 'c', width = 0.25, label = 'with regularization')
axs[1,0].bar(X + 0.25, data_svm[1], color = '#497ce3', width = 0.25,label = 'with regularization and cross-validation' )
axs[1,0].legend(loc="upper right",prop={'size': 7})
axs[1,0].set_title('SVM')


axs[1,1].bar(X + 0.00, data_mlp[0], color = 'c', width = 0.25, label = 'with regularization')
axs[1,1].bar(X + 0.25, data_mlp[1], color = '#497ce3', width = 0.25,label = 'with regularization and cross-validation' )
axs[1,1].legend(loc="upper right",prop={'size': 7})
axs[1,1].set_title('MLP')

In [ ]:
#mission 3


#SVM -w/o norm, with norm,val + w/o
data_svr = [[0.99,0.97,0.96,0.98,0.96,0.95],
[1.07,1.05,1.05,1.08,1.04,1.09],
[1.35,1.35,1.34,1.33,1.34,1.34]]

data_ann = [[1.27,1.00,0.94,1.05,1.00,1.04],
[1.10,1.08,1.07,1.02,1.03,1.12],
[1.33,1.32,1.32,1.42,1.29,1.32]]

data_gb = [[1.19,1.18,1.06,1.13,1.23,1.31],
[1.11,1.18,1.06,1.28,1.23,1.22],
[1.41,1.36,1.51,1.36,1.29,1.37]]

data_knn = [[1.13,1.13,1.16,1.10,1.15,1.13],
[2.60,2.12,1.87,2.37,1.97,2.17],
[1.48,1.50,1.44,1.51,1.37,1.40]]



X = np.arange(6)
fig, axs = plt.subplots(2, 2,figsize=(15,15))
fig.suptitle('MISSION-III Regression')
plt.setp(axs,xticks = range(len(fs_list)), xticklabels= fs_list)

for ax in axs.flat:
    ax.set( ylabel='RMSE')
    ax.tick_params(axis='x', labelrotation=45)

axs[0,0].bar(X + 0.00, data_svr[0], color = 'c', width = 0.25, label = 'without norm')
axs[0,0].bar(X + 0.25, data_svr[1], color = '#497ce3', width = 0.25,label = 'with norm' )
axs[0,0].bar(X + 0.50, data_svr[2], color = '#8649e3', width = 0.25, label = 'without norm and cross-validation')
axs[0,0].legend(loc="upper right",prop={'size': 7})
axs[0,0].set_title('SVR')

axs[0,1].bar(X + 0.00, data_ann[0], color = 'c', width = 0.25, label = 'without norm')
axs[0,1].bar(X + 0.25, data_ann[1], color = '#497ce3', width = 0.25,label = 'with norm' )
axs[0,1].bar(X + 0.50, data_ann[2], color = '#8649e3', width = 0.25, label = 'without norm and cross-validation')
axs[0,1].legend(loc="upper right",prop={'size': 7})
axs[0,1].set_title('ANN')

axs[1,0].bar(X + 0.00, data_gb[0], color = 'c', width = 0.25, label = 'without norm')
axs[1,0].bar(X + 0.25, data_gb[1], color = '#497ce3', width = 0.25,label = 'with norm' )
axs[1,0].bar(X + 0.50, data_gb[2], color = '#8649e3', width = 0.25, label = 'without norm and cross-validation')
axs[1,0].legend(loc="upper right",prop={'size': 7})
axs[1,0].set_title('Gradient Boost')

axs[1,1].bar(X + 0.00, data_knn[0], color = 'c', width = 0.25, label = 'without norm')
axs[1,1].bar(X + 0.25, data_knn[1], color = '#497ce3', width = 0.25,label = 'with norm' )
axs[1,1].bar(X + 0.50, data_knn[2], color = '#8649e3', width = 0.25, label = 'without norm and cross-validation')
axs[1,1].legend(loc="upper right",prop={'size': 7})
axs[1,1].set_title('KNN')


In [ ]:
#reg and val

#svr


data_svr = [[0.99,0.97,0.97,0.97,0.96,0.95],
[1.93,1.93,1.93,1.87,1.91,1.92]]

data_ann = [[1.22,1.02,1.00,1.00,0.99,1.10],
[2.0,1.85,1.81,2.06,1.75,1.82]]

data_ridge = [[1.01,0.96,0.96,0.96,0.96,0.98],
[1.90,1.96,1.95,1.87,1.77,1.98]]

data_lasso = [[0.86,0.86,0.86,0.86,0.86,0.86],
[1.32,2.37,2.46,2.41,1.94,2.06]]


X = np.arange(6)
fig, axs = plt.subplots(2, 2,figsize=(15,15))
fig.suptitle('MISSION-III Regression')
plt.setp(axs,xticks = range(len(fs_list)), xticklabels= fs_list)

for ax in axs.flat:
    ax.set( ylabel='RMSE')
    ax.tick_params(axis='x', labelrotation=45)

axs[0,0].bar(X + 0.00, data_svr[0], color = 'c', width = 0.25, label = 'with regularization')
axs[0,0].bar(X + 0.25, data_svr[1], color = '#497ce3', width = 0.25,label = 'with regularization and cross-validation' )
axs[0,0].legend(loc="upper right",prop={'size': 7})
axs[0,0].set_title('SVR')

axs[0,1].bar(X + 0.00, data_ann[0], color = 'c', width = 0.25, label = 'with regularization')
axs[0,1].bar(X + 0.25, data_ann[1], color = '#497ce3', width = 0.25,label = 'with regularization and cross-validation' )
axs[0,1].legend(loc="upper right",prop={'size': 7})
axs[0,1].set_title('ANN')

axs[1,0].bar(X + 0.00, data_ridge[0], color = 'c', width = 0.25, label = 'with regularization')
axs[1,0].bar(X + 0.25, data_ridge[1], color = '#497ce3', width = 0.25,label = 'with regularization and cross-validation' )
axs[1,0].legend(loc="upper right",prop={'size': 7})
axs[1,0].set_title('Ridge')

axs[1,1].bar(X + 0.00, data_lasso[0], color = 'c', width = 0.25, label = 'with regularization')
axs[1,1].bar(X + 0.25, data_lasso[1], color = '#497ce3', width = 0.25,label = 'with regularization and cross-validation')
axs[1,1].legend(loc="upper right",prop={'size': 7})
axs[1,1].set_title('Lasso')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

#logisitc reg -w/o norm, with norm, with validation 
data_lr = [[0.66,0.60,0.62,0.61,0.64,0.62],
[0.57,0.61,0.60,0.64,0.54,0.52],
[0.60,0.57,0.60,0.55,0.58,0.59]]

#SVM
data_svm = [[0.74,0.76,0.77,0.76,0.76,0.76],
[0.65,0.72,0.73,0.69,0.74,0.72],
[0.70,0.75,0.71,0.73,0.74,0.73]]

#guassian
data_gau = [[0.70,0.76,0.70,0.09,0.65,0.75],
[0.32,0.73,0.65,0.58,0.64,0.55],
[0.70,0.68,0.67,0.09,0.42,0.67]]

#Random forest
data_rf = [[0.46,0.68,0.70,0.61,0.70,0.46],
[0.46,0.68,0.72,0.65,0.70,0.49],
[0.47,0.66,0.65,0.58,0.64,0.48]]

#MLP
data_mlp = [[0.60,0.63,0.65,0.57,0.66,0.66],
[0.58,0.72,0.70,0.67,0.68,0.65],
[0.58,0.60,0.55,0.60,0.60,0.60]]

#naive bayes
data_nb = [[0.55,0.72,0.48,0.53,0.48,0.54],
[0.50,0.72,0.51,0.53,0.44,0.52],
[0.52,0.71,0.50,0.53,0.48,0.51]]

#knn
data_knn = [[0.66,0.74,0.74,0.68,0.67,0.73],
[0.28,0.58,0.51,0.32,0.42,0.28],
[0.66,0.69,0.70,0.65,0.69,0.72]]



X = np.arange(6)
fig, axs = plt.subplots(4, 2,figsize=(17,17))
fig.suptitle('MISSION-III Classification')
plt.setp(axs,xticks = range(len(fs_list)), xticklabels= fs_list)
axs[3,1].set_axis_off()
for ax in axs.flat:
    ax.set( ylabel='Macro F-1 score')
    #x.tick_params(axis='x', labelrotation=45)

axs[0,0].bar(X + 0.00, data_lr[0], color = 'c', width = 0.25, label = 'without norm')
axs[0,0].bar(X + 0.25, data_lr[1], color = '#497ce3', width = 0.25,label = 'with norm' )
axs[0,0].bar(X + 0.50, data_lr[2], color = '#8649e3', width = 0.25, label = 'without norm and cross-validation')
axs[0,0].legend(loc="upper right",prop={'size': 7})
axs[0,0].set_title('Logistic Regression')

axs[0,1].bar(X + 0.00, data_svm[0], color = 'c', width = 0.25, label = 'without norm')
axs[0,1].bar(X + 0.25, data_svm[1], color = '#497ce3', width = 0.25,label = 'with norm' )
axs[0,1].bar(X + 0.50, data_svm[2], color = '#8649e3', width = 0.25, label = 'without norm and cross-validation')
axs[0,1].legend(loc="upper right",prop={'size': 7})
axs[0,1].set_title('SVM')

axs[1,0].bar(X + 0.00, data_gau[0], color = 'c', width = 0.25, label = 'without norm')
axs[1,0].bar(X + 0.25, data_gau[1], color = '#497ce3', width = 0.25,label = 'with norm' )
axs[1,0].bar(X + 0.50, data_gau[2], color = '#8649e3', width = 0.25, label = 'without norm and cross-validation')
axs[1,0].legend(loc="upper right",prop={'size': 7})
axs[1,0].set_title('Gaussian')

axs[1,1].bar(X + 0.00, data_rf[0], color = 'c', width = 0.25, label = 'without norm')
axs[1,1].bar(X + 0.25, data_rf[1], color = '#497ce3', width = 0.25,label = 'with norm' )
axs[1,1].bar(X + 0.50, data_rf[2], color = '#8649e3', width = 0.25, label = 'without norm and cross-validation')
axs[1,1].legend(loc="upper right",prop={'size': 7})
axs[1,1].set_title('Random Forest')

axs[2,0].bar(X + 0.00, data_mlp[0], color = 'c', width = 0.25, label = 'without norm')
axs[2,0].bar(X + 0.25, data_mlp[1], color = '#497ce3', width = 0.25,label = 'with norm' )
axs[2,0].bar(X + 0.50, data_mlp[2], color = '#8649e3', width = 0.25, label = 'without norm and cross-validation')
axs[2,0].legend(loc="upper right",prop={'size': 7})
axs[2,0].set_title('MLP')

axs[2,1].bar(X + 0.00, data_nb[0], color = 'c', width = 0.25, label = 'without norm')
axs[2,1].bar(X + 0.25, data_nb[1], color = '#497ce3', width = 0.25,label = 'with norm' )
axs[2,1].bar(X + 0.50, data_nb[2], color = '#8649e3', width = 0.25, label = 'without norm and cross-validation')
axs[2,1].legend(loc="upper right",prop={'size': 7})
axs[2,1].set_title('Naive Bayes')

axs[3,0].bar(X + 0.00, data_knn[0], color = 'c', width = 0.25, label = 'without norm')
axs[3,0].bar(X + 0.25, data_knn[1], color = '#497ce3', width = 0.25,label = 'with norm' )
axs[3,0].bar(X + 0.50, data_knn[2], color = '#8649e3', width = 0.25, label = 'without norm and cross-validation')
axs[3,0].legend(loc="upper right",prop={'size': 7})
axs[3,0].set_title('KNN')


In [ ]:
#reg and val

#logistic l1 
data_l1 = [[0.58,0.61,0.59,0.62,0.64,0.61],
[0.57,0.60,0.58,0.59,0.60,0.55]]

#logistic l2
data_l2 = [[0.66,0.62,0.63,0.61,0.64,0.62],
[0.60,0.58,0.62,0.55,0.58,0.58]]

#SVM
data_svm = [[0.72,0.74,0.70,0.75,0.76,0.75],
[0.72,0.74,0.75,0.71,0.73,0.71]]

#MLP
data_mlp = [[0.60,0.61,0.61,0.57,0.63,0.65],
[0.58,0.60,0.60,0.60,0.60,0.59]]


X = np.arange(6)
fig, axs = plt.subplots(2, 2,figsize=(15,15))
fig.suptitle('MISSION-III Classification')
plt.setp(axs,xticks = range(len(fs_list)), xticklabels= fs_list)

for ax in axs.flat:
    ax.set( ylabel='Macro F-1 score')
    ax.tick_params(axis='x', labelrotation=45)

axs[0,0].bar(X + 0.00, data_l1[0], color = 'c', width = 0.25, label = 'with regularization')
axs[0,0].bar(X + 0.25, data_l1[1], color = '#497ce3', width = 0.25,label = 'with regularization and cross-validation' )
axs[0,0].legend(loc="upper right",prop={'size': 7})
axs[0,0].set_title('Logistic Regression L1')

axs[0,1].bar(X + 0.00, data_l2[0], color = 'c', width = 0.25, label = 'with regularization')
axs[0,1].bar(X + 0.25, data_l2[1], color = '#497ce3', width = 0.25,label = 'with regularization and cross-validation' )
axs[0,1].legend(loc="upper right",prop={'size': 7})
axs[0,1].set_title('Logistic Regression L2')

axs[1,0].bar(X + 0.00, data_svm[0], color = 'c', width = 0.25, label = 'with regularization')
axs[1,0].bar(X + 0.25, data_svm[1], color = '#497ce3', width = 0.25,label = 'with regularization and cross-validation' )
axs[1,0].legend(loc="upper right",prop={'size': 7})
axs[1,0].set_title('SVM')


axs[1,1].bar(X + 0.00, data_mlp[0], color = 'c', width = 0.25, label = 'with regularization')
axs[1,1].bar(X + 0.25, data_mlp[1], color = '#497ce3', width = 0.25,label = 'with regularization and cross-validation' )
axs[1,1].legend(loc="upper right",prop={'size': 7})
axs[1,1].set_title('MLP')